In [ ]:
import numpy as np
import chardet
import joblib
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt 
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.io as pio
import plotly.colors as pc

%matplotlib inline  
import psutil
from pathlib import Path
#from Functions import *
from scipy.io import loadmat
import glob
from scipy.signal import savgol_filter
from tqdm import tqdm
from sysidentpy.model_structure_selection import FROLS
from sysidentpy.basis_function import Polynomial
from sysidentpy.metrics import mean_squared_error
import time
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import os

# Functions

In [ ]:

import pandas as pd
import numpy as np

def build_arx_lagged_with_scalers(
    df,
    input_cols,
    output_cols,
    scaler_X_func,
    scaler_y_func,
    na=0,
    nb_past=0,
    nf_future=0,
    test_name_col='test_name',
    y_initial_mode='original'  # 'original' ➔ normal; 'zero' ➔ prepend zero rows after scaling
):
    """
    Build ARX lagged data from a single DataFrame using custom scaler functions.
    
    Parameters:
    - df: original DataFrame (raw, unscaled) with test_name column
    - input_cols: list of input column names to scale
    - output_cols: list of output column names to scale
    - scaler_X_func: function or fitted scaler to scale inputs (df ➔ df)
    - scaler_y_func: function or fitted scaler to scale outputs (df ➔ df)
    - na: number of output lags (autoregressive)
    - nb_past: number of past input lags (exogenous input past)
    - nf_future: number of future input lags (exogenous input preview/future)
    - test_name_col: column name that identifies test cases
    - y_initial_mode: 'original' ➔ normal; 'zero' ➔ prepend zero rows after scaling

    Returns:
    - X_lagged_df: lagged input DataFrame
    - y_target_df: target output DataFrame
    - y_initial_df: DataFrame of initial output values used as initial conditions
    """

    X_df_list = []
    Y_df_list = []
    Y_initial_list = []

    # Unique test cases
    test_names = df[test_name_col].unique()

    for test in test_names:
        df_test = df[df[test_name_col] == test].copy()

        # Apply scaling functions first
        X_scaled = scaler_X_func(df_test[input_cols])
        y_scaled = scaler_y_func(df_test[output_cols])

        # Convert to DataFrames and keep test_name column for tracking
        X_scaled_df = pd.DataFrame(X_scaled, columns=input_cols)
        y_scaled_df = pd.DataFrame(y_scaled, columns=output_cols)

        if y_initial_mode == 'zero':
            # Create zero rows in scaled space
            lag_required = max(na, nb_past)

            zero_inputs = pd.DataFrame(
                np.zeros((lag_required, len(input_cols))),
                columns=input_cols
            )

            zero_outputs = pd.DataFrame(
                np.zeros((lag_required, len(output_cols))),
                columns=output_cols
            )

            # Concatenate zero inputs/outputs
            zero_rows_inputs = zero_inputs
            zero_rows_outputs = zero_outputs

            # Concatenate zero rows on top of X_scaled_df and y_scaled_df
            X_scaled_df = pd.concat([zero_rows_inputs, X_scaled_df], ignore_index=True)
            y_scaled_df = pd.concat([zero_rows_outputs, y_scaled_df], ignore_index=True)

        # Now proceed with lag creation
        inputs = X_scaled_df.values
        outputs = y_scaled_df.values

        n_samples = len(X_scaled_df)

        lag_required = max(na, nb_past)
        min_future_offset = nf_future

       
        # ➤ Save the first `na` rows as y_initial
        if y_initial_mode == 'zero':
            start_idx = lag_required
        else:  # 'original'
            start_idx = 0

        end_idx = n_samples - nf_future
        
        if na > 0:
            if y_initial_mode == 'original':
               
                y_initial = y_scaled_df.iloc[:lag_required].copy()
            else:  # 'zero'
                y_initial= pd.DataFrame(np.zeros((na, len(output_cols))), columns=output_cols)
               
                
            y_initial[test_name_col] = test
            Y_initial_list.append(y_initial)

        X_rows = []
        Y_rows = []

        for t in range(start_idx, end_idx):
            row = {}
            
            #Add current unscaled input values
            for i, in_col in enumerate(input_cols):
                row[in_col] = inputs[t, i]  # Current time step value

            # Add output past lags (autoregressive)
            for lag in range(1, na + 1):
                for i, out_col in enumerate(output_cols):
                    idx = t - lag
                    if y_initial_mode == 'zero' and idx < 0:
                        row[f'{out_col}_lag_{lag}'] = 0.0
                    else:
                        row[f'{out_col}_lag_{lag}'] = outputs[idx, i]
                            
            # Add input past lags (exogenous)
            for lag in range(1, nb_past + 1):
                for i, in_col in enumerate(input_cols):
                    idx = t - lag
                    if y_initial_mode == 'zero' and idx < 0:
                        row[f'{in_col}_past_{lag}'] = 0.0
                    else:
                        row[f'{in_col}_past_{lag}'] = inputs[idx, i]

            # Add input future lags (preview control)
            for lag in range(1, nf_future + 1):
                if t + lag < n_samples:
                    for i, in_col in enumerate(input_cols):
                        row[f'{in_col}_future_{lag}'] = inputs[t + lag, i]
                else:
                    for in_col in input_cols:
                        row[f'{in_col}_future_{lag}'] = 0.0  # optional padding if beyond end

            X_rows.append(row)
            Y_rows.append(outputs[t])

        # Create DataFrames for this test
        X_df = pd.DataFrame(X_rows)
        Y_df = pd.DataFrame(Y_rows, columns=output_cols)

        # Add test_name to track
        X_df[test_name_col] = test
        Y_df[test_name_col] = test

        X_df_list.append(X_df)
        Y_df_list.append(Y_df)

    # Combine all test cases into final DataFrames
    X_lagged_df = pd.concat(X_df_list, ignore_index=True)
    y_target_df = pd.concat(Y_df_list, ignore_index=True)

    if na > 0:
        y_initial_df = pd.concat(Y_initial_list, ignore_index=True)
    else:
        y_initial_df = pd.DataFrame()

    return X_lagged_df, y_target_df, y_initial_df


In [ ]:

def predict_recursive_series(model, X_df, output_cols, X_feature_names, na):
    """
    Predict recursively one step at a time using lag updates.

    Parameters
    ----------
    model : Trained model
    X_df : pd.DataFrame
        Input features (with lag columns, same format as training)
    output_cols : list of str
        List of output columns (e.g., ['heave', 'pitch', ...])
    X_feature_names : list
        Ordered list of input feature names expected by the model
    na : int
        Number of output lags

    Returns
    -------
    y_pred_df : pd.DataFrame
        Recursive prediction results (same shape as y_df)
    x_used_df : pd.DataFrame
        Input rows actually used at each timestep (after lag updates)
    """
    import numpy as np
    import pandas as pd

    if na == 0:
        y_pred = model.predict(X_df)
        return pd.DataFrame(y_pred, columns=output_cols), X_df.copy()

    n_steps = len(X_df)
    n_outputs = len(output_cols)

    y_pred = np.zeros((n_steps, n_outputs))
    x_used_rows = []

    # Start with the first input row
    x_row = X_df.iloc[0].copy()
    
    value_threshold = 5

    for t in range(n_steps):
        
        # Check if this row contains any absurd value
        if np.any(np.abs(x_row.values) > value_threshold):
            print(f"[ABORT] Found large value at step t={t}. Aborting prediction.")
            return pd.DataFrame(np.zeros((n_steps, n_outputs)), columns=output_cols), pd.DataFrame(x_used_rows)
        
        # Build input feature vector for model
        x_input = pd.DataFrame([[x_row[feat] for feat in X_feature_names]], columns=X_feature_names)

        # Save the input row used at this step
        x_used_rows.append(x_input.iloc[0])

        # Predict
        pred = model.predict(x_input)
        # Ensure it's always a 2D shape: (1, n_outputs)
        if pred.ndim == 1:
            pred = pred.reshape(1, -1)

        pred = pred[0]  # Now this is always indexable

        y_pred[t] = pred

        if t < n_steps - 1:
            # Prepare next input row
            x_row_next = X_df.iloc[t + 1].copy()

            for col in output_cols:
                # Shift lags: lag_n = lag_{n-1}, ..., lag_2 = lag_1
                for lag in reversed(range(2, na + 1)):
                    x_row_next[f"{col}_lag_{lag}"] = x_row[f"{col}_lag_{lag - 1}"]
                # Set lag_1 to current prediction
                x_row_next[f"{col}_lag_1"] = pred[output_cols.index(col)]

            x_row = x_row_next.copy()

    y_pred_df = pd.DataFrame(y_pred, columns=output_cols)
    x_used_df = pd.DataFrame(x_used_rows)

    return y_pred_df, x_used_df


In [ ]:
def select_input_features(
    X_lagged_df,
    input_type='eta',
    keep_future=True,
    keep_past=True
):
    """
    Select input columns, preserving:
    - Base inputs (e.g., 'eta', 'eta_velocity')
    - Their lagged versions ('_past_', '_future_', '_lag_')
    - Autoregressive terms ('_lag_')
    - 'test_name' (if present)
    """
    cols_to_keep = []

    # Base input variable(s) (always keep these)
    if input_type == 'eta':
        base_inputs = ['eta']
    elif input_type == 'eta+vel':
        base_inputs = ['eta', 'eta_velocity']
    elif input_type == 'eta+vel+acc':
        base_inputs = ['eta', 'eta_velocity', 'eta_acceleration']
    else:
        raise ValueError("input_type must be 'eta', 'eta+vel', or 'eta+vel+acc'")

    
    # Step 1: Add base inputs (if they exist in the DataFrame)
    for col in base_inputs:
        if col in X_lagged_df.columns and col not in cols_to_keep:
            cols_to_keep.append(col)

    # Step 2: Add lagged versions (past/future) of base inputs
    for col in base_inputs:
        if keep_past:
            past_cols = [c for c in X_lagged_df.columns if f"{col}_past_" in c]
            cols_to_keep.extend(past_cols)
        if keep_future:
            future_cols = [c for c in X_lagged_df.columns if f"{col}_future_" in c]
            cols_to_keep.extend(future_cols)

    # Step 3: Add autoregressive terms (if any)
    arx_cols = [c for c in X_lagged_df.columns if '_lag_' in c and c not in cols_to_keep]
    cols_to_keep.extend(arx_cols)

    # Step 4: Always keep 'test_name' (if present)
    test_name_col = next((c for c in X_lagged_df.columns if c.lower() == 'test_name'), None)
    if test_name_col and test_name_col not in cols_to_keep:
        cols_to_keep.append(test_name_col)

    # Step 5: Fallback if no features found (return all columns + warning)
    if not cols_to_keep:
        print("Warning: No matching features found. Returning all columns.")
        return X_lagged_df.copy()

    return X_lagged_df[cols_to_keep].copy()

# loading Data

In [ ]:
# load data
df_train_full = pd.read_csv('prepared_data/train_data.csv')
df_val_full = pd.read_csv('prepared_data/val_data.csv')
df_test_full = pd.read_csv('prepared_data/test_data.csv')

print(df_train_full.head())
print(df_val_full.head())
print(df_test_full.head())


In [ ]:
# Extrating Only the case we want to work with
case='Tp6p8s_Hs2m'
df_case_train = df_train_full[df_train_full['test_name'] == case].copy().reset_index(drop=True)
df_case_val = df_val_full[df_val_full['test_name'] == case].copy().reset_index(drop=True)
df_case_test = df_test_full[df_test_full['test_name'] == case].copy().reset_index(drop=True)



In [ ]:
df_case_train

# Scaling Data

In [ ]:
from sklearn.preprocessing import MinMaxScaler


input_cols = ['eta', 'eta_velocity', 'eta_acceleration']
output_cols = ['heave',  'pitch', 'pendulum']

# Initialize scalers
scaler_X = MinMaxScaler(feature_range=(-1, 1))
scaler_y = MinMaxScaler(feature_range=(-1, 1))

# Fit on training data (DataFrames, not NumPy arrays)
scaler_X.fit(df_train_full[input_cols])
scaler_y.fit(df_train_full[output_cols])

# Transform train
X_train_scaled = scaler_X.transform(df_train_full[input_cols])
y_train_scaled = scaler_y.transform(df_train_full[output_cols])

X_train_scaled_df = pd.DataFrame(X_train_scaled, columns=input_cols)
y_train_scaled_df = pd.DataFrame(y_train_scaled, columns=output_cols)

X_train_scaled_df['test_name'] = df_train_full['test_name'].values
y_train_scaled_df['test_name'] = df_train_full['test_name'].values

# Repeat for validation (no fit!)
X_val_scaled = scaler_X.transform(df_val_full[input_cols])
y_val_scaled = scaler_y.transform(df_val_full[output_cols])

X_val_scaled_df = pd.DataFrame(X_val_scaled, columns=input_cols)
y_val_scaled_df = pd.DataFrame(y_val_scaled, columns=output_cols)

X_val_scaled_df['test_name'] = df_val_full['test_name'].values
y_val_scaled_df['test_name'] = df_val_full['test_name'].values

# Define the scaling functions (they apply the .transform)
scaler_X_func = lambda df: scaler_X.transform(df)
scaler_y_func = lambda df: scaler_y.transform(df)

In [ ]:
input_cols = ['eta', 'eta_velocity', 'eta_acceleration']
output_cols = ['heave',  'pitch', 'pendulum']
# Initialize scalers
scaler_X_nov = MinMaxScaler(feature_range=(-1, 1))
scaler_y_nov = MinMaxScaler(feature_range=(-1, 1))

# Fit on training data (DataFrames, not NumPy arrays)
scaler_X_nov.fit(df_train_full[input_cols])
scaler_y_nov.fit(df_train_full[output_cols])





# define scalling function
# Define the scaling functions (they apply the .transform)
scaler_X_func_nov = lambda df: scaler_X_nov.transform(df)
scaler_y_func_nov = lambda df: scaler_y_nov.transform(df)

In [ ]:
output_cols = ['heave']
# Initialize scalers
scaler_X_nov = MinMaxScaler(feature_range=(-1, 1))
scaler_y_heave = MinMaxScaler(feature_range=(-1, 1))

# Fit on training data (DataFrames, not NumPy arrays)
scaler_X_nov.fit(df_train_full[input_cols])
scaler_y_heave.fit(df_train_full[output_cols])





# define scalling function
# Define the scaling functions (they apply the .transform)
scaler_y_func_heave = lambda df: scaler_y_heave.transform(df)

In [ ]:
output_cols = ['pitch']
# Initialize scalers

scaler_y_pitch = MinMaxScaler(feature_range=(-1, 1))

# Fit on training data (DataFrames, not NumPy arrays)

scaler_y_pitch.fit(df_train_full[output_cols])

# define scalling function
# Define the scaling functions (they apply the .transform)

scaler_y_func_pitch = lambda df: scaler_y_pitch.transform(df)

In [ ]:
output_cols = ['pendulum']
# Initialize scalers

scaler_y_pend = MinMaxScaler(feature_range=(-1, 1))

# Fit on training data (DataFrames, not NumPy arrays)

scaler_y_pend.fit(df_train_full[output_cols])

# define scalling function
# Define the scaling functions (they apply the .transform)

scaler_y_func_pend = lambda df: scaler_y_pend.transform(df)

# Lag analysis

# ACF and PACF

In [ ]:
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from sklearn.feature_selection import mutual_info_regression
import numpy as np

def automated_lag_analysis(
    df,
    test_case,
    output_cols,
    max_lag=40
):
    """
    Automated lag analysis for ACF, PACF, and MI for multiple output columns.

    Parameters:
    - df: your training DataFrame (with 'test_name' column)
    - test_case: specific test_name or 'all' for the entire dataset
    - output_cols: list of output columns to process
    - max_lag: number of lags to compute
    """

    # Filter the data for the specific test case, or use all
    if test_case == 'all':
        df_case = df.copy()
        case_label = 'ALL_CASES'
    else:
        df_case = df[df['test_name'] == test_case]
        case_label = test_case

    for col in output_cols:
        print(f"\n=== Lag Analysis for {col} (Test Case: {case_label}) ===")

        # Extract the signal
        y_signal = df_case[col].values

        # Prepare MI scores
        mi_scores = []

        for lag in range(1, max_lag + 1):
            y_t = y_signal[lag:]
            y_t_minus_lag = y_signal[:-lag]

            if len(y_t_minus_lag) > 0:
                mi = mutual_info_regression(
                    y_t_minus_lag.reshape(-1, 1),
                    y_t.ravel(),
                    random_state=42
                )
                mi_scores.append(mi[0])
            else:
                mi_scores.append(0)

        # Plot ACF, PACF, MI side by side
        fig, axs = plt.subplots(1, 3, figsize=(18, 4))

        # ACF
        plot_acf(y_signal, lags=max_lag, ax=axs[0])
        axs[0].set_title(f'ACF - {col}')

        # PACF
        plot_pacf(y_signal, lags=max_lag, ax=axs[1])
        axs[1].set_title(f'PACF - {col}')

        # Mutual Information
        axs[2].plot(range(1, max_lag + 1), mi_scores, marker='o')
        axs[2].set_title(f'Mutual Information - {col}')
        axs[2].set_xlabel('Lag')
        axs[2].set_ylabel('MI')
        axs[2].grid(True)

        plt.suptitle(f'Lag Analysis - {col} - {case_label}', fontsize=14)
        plt.tight_layout()
        plt.show()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

sns.set(style="whitegrid", context="talk")

def automated_lag_analysis_improved(
    df,
    test_case,
    output_cols,
    max_lag=40
):
    """
    Automated lag analysis for ACF and PACF for multiple output columns.

    Parameters:
    - df: your training DataFrame (with 'test_name' column)
    - test_case: specific test_name or 'all' for the entire dataset
    - output_cols: list of output columns to process
    - max_lag: number of lags to compute
    """

    # Filter the data for the specific test case, or use all
    if test_case == 'all':
        df_case = df.copy()
        case_label = 'ALL_CASES'
    else:
        df_case = df[df['test_name'] == test_case]
        case_label = test_case

    for col in output_cols:
        print(f"\n=== Lag Analysis for {col} (Test Case: {case_label}) ===")

        # Extract the signal
        y_signal = df_case[col].values

        # Create larger figure and improved layout
        fig, axs = plt.subplots(1, 2, figsize=(20, 6))

        # ACF
        plot_acf(y_signal, lags=max_lag, ax=axs[0])
        axs[0].set_title(f'ACF - {col}')
        axs[0].set_ylim(-0.5, 1.2)
        axs[0].set_xlabel("$n_a")

        # PACF
        plot_pacf(y_signal, lags=max_lag, ax=axs[1])
        axs[1].set_title(f'PACF - {col}')
        axs[1].set_ylim(-1.2, 1.2)
        axs[1].set_xlabel("$_a$")

        plt.suptitle(f'Output Lag Analysis - {col} ', fontsize=18, weight='bold')
        plt.tight_layout(rect=[0, 0.03, 1, 0.95])
        plt.show()


In [ ]:
automated_lag_analysis_improved(
    df=df_train_full,
    test_case='Tp6p8s_Hs2m',
    output_cols=['heave',  'pitch',  'pendulum'],
    max_lag=35
)


## MI and Correlation 

In [ ]:
case='Tp6p8s_Hs2m'

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn.feature_selection import mutual_info_regression

sns.set(style="whitegrid", context="notebook")

def input_output_mi_heatmap_by_lag(
    df,
    test_case,
    input_cols,
    output_cols,
    max_lag=10  # include nb0 to nb{max_lag}
):
    df_case = df.copy() if test_case == 'all' else df[df['test_name'] == test_case]
    case_label = 'ALL_CASES' if test_case == 'all' else test_case

    X_data = df_case[input_cols].values
    Y_data = df_case[output_cols].values
    n_samples = len(df_case)

    for input_idx, input_col in enumerate(input_cols):
        mi_matrix = []

        for output_idx in range(len(output_cols)):
            mi_row = []

            for lag in range(0, max_lag + 1):  # now starts from lag = 0 (nb0)
                if lag >= n_samples:
                    mi_row.append(0)
                    continue

                if lag == 0:
                    x = X_data[:, input_idx]
                    y = Y_data[:, output_idx]
                else:
                    x = X_data[:-lag, input_idx]
                    y = Y_data[lag:, output_idx]

                mi = mutual_info_regression(x.reshape(-1, 1), y.ravel(), random_state=42)
                mi_row.append(mi[0])

            mi_matrix.append(mi_row)

        # Build DataFrame: outputs as rows, lags as columns
        lag_labels = [f'nb{lag}' for lag in range(0, max_lag + 1)]
        mi_df = pd.DataFrame(mi_matrix, index=output_cols, columns=lag_labels)

        # Plot heatmap
        plt.figure(figsize=(max_lag * 0.8 + 4, len(output_cols) * 0.8 + 2))
        ax = sns.heatmap(
            mi_df,
            annot=True,
            fmt=".3f",
            cmap="YlOrRd",
            cbar_kws={"label": "Mutual Information "},
            linewidths=0.5
        )

        ax.set_title(f'Mutual Information Between Output DOFs and Free Surface Elevation (η) ', fontsize=16, pad=20)
        ax.set_xlabel("Lag ($n_b$)", fontsize=13)
        ax.set_ylabel("Outputs", fontsize=13)
        
        ax.tick_params(labelsize=12)
        plt.tight_layout()
        plt.show()


In [ ]:
input_output_mi_heatmap_by_lag(
    df=df_train_full,
    test_case=case,
    input_cols=['eta'],
    output_cols=['heave',  'pitch',  'pendulum'],
    max_lag=15
)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

sns.set(style="whitegrid", context="notebook")

def input_output_corr_heatmap_by_lag(
    df,
    test_case,
    input_cols,
    output_cols,
    max_lag=10  # includes nb0 to nb{max_lag}
):
    df_case = df.copy() if test_case == 'all' else df[df['test_name'] == test_case]
    case_label = 'ALL_CASES' if test_case == 'all' else test_case

    X_data = df_case[input_cols].values
    Y_data = df_case[output_cols].values
    n_samples = len(df_case)

    for input_idx, input_col in enumerate(input_cols):
        corr_matrix = []

        for output_idx in range(len(output_cols)):
            corr_row = []

            for lag in range(0, max_lag + 1):  # nb0 to nb{max_lag}
                if lag >= n_samples:
                    corr_row.append(np.nan)
                    continue

                if lag == 0:
                    x = X_data[:, input_idx]
                    y = Y_data[:, output_idx]
                else:
                    x = X_data[:-lag, input_idx]
                    y = Y_data[lag:, output_idx]

                corr = np.corrcoef(x, y)[0, 1]
                corr_row.append(corr)

            corr_matrix.append(corr_row)

        lag_labels = [f'nb{lag}' for lag in range(0, max_lag + 1)]
        corr_df = pd.DataFrame(corr_matrix, index=output_cols, columns=lag_labels)

        # Plot heatmap
        plt.figure(figsize=(max_lag * 0.8 + 4, len(output_cols) * 0.8 + 2))
        ax = sns.heatmap(
            corr_df,
            annot=True,
            fmt=".2f",
            cmap="coolwarm",
            center=0,
            cbar_kws={"label": "Correlation "},
            linewidths=0.5,
            vmin=-1,
            vmax=1
        )

        ax.set_title(f'Correlation Heatmap Between Output DOFs and Free Surface Elevation (η) ', fontsize=16, pad=20)
        ax.set_xlabel("Lag ($n_b$)", fontsize=13)
        ax.set_ylabel("Outputs", fontsize=13)
        ax.tick_params(axis='x', labelsize=12)
        ax.tick_params(axis='y', labelsize=12)
        plt.tight_layout()
        plt.show()


In [ ]:
input_output_corr_heatmap_by_lag(
    df=df_train_full,
    test_case=case,
    input_cols=['eta'],
    output_cols=['heave',  'pitch',  'pendulum'],
    max_lag=15
)

## Checking if the system is causual or not


In [ ]:
case='Tp6p8s_Hs2m'
input_cols = ['eta', 'eta_velocity', 'eta_acceleration']
output_cols = ['heave',  'pitch', 'pendulum']

In [ ]:
# Initialize empty DataFrames before the loop
metrics_df_nd = pd.DataFrame()

In [ ]:
import os
# Base save folder
save_folder = 'linear_regression/causal_or_not/saved_models'

# Create the directory if it doesn't exist
os.makedirs(save_folder, exist_ok=True)

In [ ]:
# define loop values 
na  = 0
nb  =4 
nf_max=6
degree = 1
xcase = 'eta'


In [ ]:

for nf in range(0,nf_max+1): 
    


    # Create the pipeline model
    model = Pipeline([
            ('polynomial', PolynomialFeatures(degree=degree, interaction_only=False)),
            ('linear', LinearRegression())
        ])
        
    # prepare training data 
    print(f'-----Preprocessing case of na={str(na)} ,nb={str(nb)} and nf={str(nf)} , with xcase: {xcase}----')
    # get  df at true scale with only this test case
    df_case_train=df_train_full[df_train_full['test_name']==case].reset_index(drop=True)
    
    

    dfx_train,dfy_train,yi_train = build_arx_lagged_with_scalers(
            df = df_case_train,
            input_cols  = input_cols,
            output_cols   = output_cols,
            scaler_X_func   = scaler_X_func,
            scaler_y_func   = scaler_y_func,
            na=na,
            nb_past=nb,
            nf_future=nf,
            test_name_col='test_name',
            y_initial_mode='zero'  # 'original' ➔ skip initial rows, 'zero' ➔ pad lags with zeros
    )
        
    y_train_target = dfy_train[output_cols].reset_index(drop=True)



    # prepare validation data
    df_case_val=df_val_full[df_val_full['test_name']==case].reset_index(drop=True)

    dfx_val,dfy_val,yi_val = build_arx_lagged_with_scalers(
            df = df_case_val,
            input_cols  = input_cols,
            output_cols   = output_cols,
            scaler_X_func   = scaler_X_func,
            scaler_y_func   = scaler_y_func,
            na=na,
            nb_past=nb,
            nf_future=nf,
            test_name_col='test_name',
            y_initial_mode='zero'  # 'original' ➔ skip initial rows, 'zero' ➔ pad lags with zeros
    )
    print(f"nf={nf}, nd={nd}, X_train shape: {X_train_selected_df.shape}, y shape: {y_train_target.shape}")


    # select eta only in input features
    X_train_selected_df = select_input_features(
        dfx_train.drop(columns='test_name'),
        input_type=xcase,
        keep_future=True,
        keep_past=True
    )

    X_val_selected_df = select_input_features(
        dfx_val.drop(columns='test_name'),
        input_type=xcase,
        keep_future=True,
        keep_past=True
    )

    if nf == 0:
        nd_list = [0, 1, 2, 3, 4, 5 , 6, 7, 8]  # Include all lags from 0 to 8
    else:
        nd_list = [0] 
        
    for nd in nd_list:
        if nd > 0:
        # Filter out eta and its past lags based on nd
            X_train_selected_df = X_train_selected_df.loc[:, [ col for col in X_train_selected_df.columns
                if not (col == 'eta' and nd > 0) and not (col.startswith('eta_past_') and int(col.split('_')[-1]) < nd)
            ]]
            X_val_selected_df = X_val_selected_df.loc[:, [ col for col in X_val_selected_df.columns
                if not (col == 'eta' and nd > 0) and not (col.startswith('eta_past_') and int(col.split('_')[-1]) < nd)
            ]]           
        # Model parameters
        model_name = F'LinearRegression_' + xcase + '_na' + str(na) + '_nb' + str(nb) + '_nf' + str(nf) + '_nd' + str(nd) + '_degree' + str(degree) + '.joblib'
        # ============================
        # MEASURE CPU & MEMORY USAGE
        # ============================
        print(f'-----Training model----')
        process = psutil.Process()
        if X_train_selected_df.empty or X_train_selected_df.shape[1] == 0:
            print(f"[SKIP] No input features left for nf={nf}, nd={nd}")
            continue

        # Train the model
        #start_time = time.perf_counter()
        model.fit(X_train_selected_df  ,y_train_target  )
        #train_time = time.perf_counter() - start_time
        #memory_usage_train = process.memory_info().rss / (1024 * 1024)  # MB

        print(f'-----predicting on training and validation data----')
        # Predict on train and validation data
        #start_time = time.perf_counter()
        y_pred_train_scaled = model.predict(X_train_selected_df)
        y_pred_val_scaled = model.predict(X_val_selected_df)

        # Inverse transform to original scale
        y_pred_train = scaler_y.inverse_transform(y_pred_train_scaled)
        y_pred_val = scaler_y.inverse_transform(y_pred_val_scaled)

        #predict_time = time.perf_counter() - start_time
        memory_usage_predict = process.memory_info().rss / (1024 * 1024)  # MB

        # Convert predictions to DataFrames for easier handling
        y_pred_train_df = pd.DataFrame(y_pred_train, columns=output_cols)
        y_pred_val_df = pd.DataFrame(y_pred_val, columns=output_cols)


        # Get true values aligned with dfy_train and dfy_val indexes
        y_true_train =scaler_y.inverse_transform(dfy_train[output_cols].reset_index(drop=True))
        y_true_train_df = pd.DataFrame(y_true_train, columns=output_cols)
        
        y_true_val = scaler_y.inverse_transform(dfy_val[output_cols].reset_index(drop=True))
        y_true_val_df = pd.DataFrame(y_true_val, columns=output_cols)
        

        
        # save model info to dfs
        metrics_row = {
        'model_name': model_name,
        'na': na,
        'nb': nb,
        'nf': nf,
        'nd' : nd,
        'xcase': xcase,
        'test case': case,
    }

        perf_row = {
        'model_name': model_name,
        'na': na,
        'nb': nb,
        'nf': nf,
        'degree': degree,
        'xcase': xcase,
        'train_time': train_time,
        'train_memory_MB': memory_usage_train,
        'predict_time': predict_time,
        'predict_memory_MB': memory_usage_predict
            }
        print(f'-----Eavluating model----')
        # Compute metrics
        for col in output_cols:
            mse_train = mean_squared_error(y_true_train_df[col], y_pred_train_df[col])
            r2_train = r2_score(y_true_train_df[col], y_pred_train_df[col])

            mse_val = mean_squared_error(y_true_val_df[col], y_pred_val_df[col])
            r2_val = r2_score(y_true_val_df[col], y_pred_val_df[col])

            # Add to metrics row
            metrics_row[f'r2_train_{col}'] = r2_train
            metrics_row[f'mse_train_{col}'] = mse_train
            metrics_row[f'r2_val_{col}'] = r2_val
            metrics_row[f'mse_val_{col}'] = mse_val
        
        print("metrics_row:", metrics_row)
        print("perf_row:", perf_row)
        # Append the row dictionaries as new rows in the DataFrames
        metrics_df_nd = pd.concat([metrics_df_nd, pd.DataFrame([metrics_row])], ignore_index=True)
        #perf_df = pd.concat([perf_df, pd.DataFrame([perf_row])], ignore_index=True)
        

In [ ]:
metrics_df_nd.loc[metrics_df_nd['nf'] > 0, 'nd'] = -metrics_df_nd['nf']

metrics_df_nd['nd'].unique()  # Ensure nd is treated as a categorical variable

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

sns.set(style="whitegrid", context="talk")

# Prepare the data
plot_df = metrics_df_nd[['nd', 'r2_train_heave', 'r2_train_pitch', 'r2_train_pendulum']].copy()
plot_df = plot_df.sort_values('nd')
plot_df = plot_df[plot_df['nd'] != 8]

# Create the plot
fig, ax = plt.subplots(figsize=(10, 7))

# Plot R² lines
ax.plot(plot_df['nd'], plot_df['r2_train_heave'], marker='o', label='Heave')
ax.plot(plot_df['nd'], plot_df['r2_train_pitch'], marker='s', label='Pitch')
ax.plot(plot_df['nd'], plot_df['r2_train_pendulum'], marker='^', label='Pendulum')

# Move y-axis to x=0
ax.spines['left'].set_position(('data', 0))
ax.yaxis.set_label_position("left")
ax.yaxis.tick_left()

# Place y-axis label at the top
ax.set_ylabel('')  # remove it

# Manually add y-axis label at x=0
ax.text(
    0, 1.02, 'R²', transform=ax.transData,
    ha='right', va='bottom', fontsize=13
)
# Move y-axis to x = 0
ax.spines['left'].set_position(('data', 0))
ax.yaxis.set_label_position("left")
ax.yaxis.tick_left()
# Clean up right and top spines
ax.spines['right'].set_color('none')
ax.spines['top'].set_color('none')

# Set X ticks every 1 unit
min_nd = plot_df['nd'].min()
max_nd = plot_df['nd'].max()
xticks = np.arange(min_nd, max_nd + 1, 1)

ax.set_xticks(xticks)


# Set Y limits and grid
ax.set_ylim(0, 1)
yticks = ax.get_yticks()
yticks = [y for y in yticks if y != 0]
ax.set_yticks(yticks)


ax.set_xlabel('$n_d$')
ax.legend()

# Title with more spacing
ax.set_title('Effect of $n_d$ on Training R² — $n_b$ = 4, $n_a$ = 0', pad=50)


# Enable both vertical and horizontal gridlines at tick locations
ax.grid(False, which='both', axis='both')

plt.tight_layout()
plt.show()


# Checking model in parallel configration For best Input Fearures

In [ ]:
# Base save folder
save_folder = 'ARX/parralel_config'

# Create the directory if it doesn't exist
os.makedirs(save_folder, exist_ok=True)


Adjust output_col , xcase, and scaler functions according to the desired case.

In [ ]:
nb_max=[0,1,2,3,4,5,6,7]
nf_max=[0,1,2,3,4,5,6,7]
degree=1
na=2
case='Tp6p8s_Hs2m'
output_cols = ['pendulum'] 
input_cols=['eta' , 'eta_velocity', 'eta_acceleration']


In [ ]:
# Initialize empty DataFrames before the loop
metrics_df_ARX_pendulum_only_parralel_eta_only = pd.DataFrame()


xcase='eta'

In [ ]:
for nb in nb_max:
    for nf in nf_max:
        

        # Create the pipeline model
        model = Pipeline([
                ('polynomial', PolynomialFeatures(degree=degree, interaction_only=False)),
                ('linear', LinearRegression())
            ])
            
        # prepare training data 
        print(f'-----Preprocessing case of na={str(na)} ,nb={str(nb)} and nf={str(nf)} , with xcase: {xcase}----')
        # get  df at true scale witAh only this test case
        df_case_train=df_train_full[df_train_full['test_name']==case].reset_index(drop=True)



        dfx_train,dfy_train,yi_train = build_arx_lagged_with_scalers(
                df = df_case_train,
                input_cols  = input_cols,
                output_cols   = output_cols,
                scaler_X_func   = scaler_X_func_nov,
                scaler_y_func   = scaler_y_func_pend,
                na=na,
                nb_past=nb,
                nf_future=nf,
                test_name_col='test_name',
                y_initial_mode='zero'  # 'original' ➔ skip initial rows, 'zero' ➔ pad lags with zeros
        )
            
        y_train_target = dfy_train[output_cols].reset_index(drop=True)



        # prepare validation data
        df_case_val=df_val_full[df_val_full['test_name']==case].reset_index(drop=True)

        dfx_val,dfy_val,yi_val = build_arx_lagged_with_scalers(
                df = df_case_val,
                input_cols  = input_cols,
                output_cols   = output_cols,
                scaler_X_func   = scaler_X_func_nov,
                scaler_y_func   = scaler_y_func_pend,
                na=na,
                nb_past=nb,
                nf_future=nf,
                test_name_col='test_name',
                y_initial_mode='original'  # 'original' ➔ skip initial rows, 'zero' ➔ pad lags with zeros
        )


        # select eta only in input features
        X_train_selected_df = select_input_features(
            dfx_train.drop(columns='test_name'),
            input_type=xcase,
            keep_future=True,
            keep_past=True
        )

        X_val_selected_df = select_input_features(
            dfx_val.drop(columns='test_name'),
            input_type=xcase,
            keep_future=True,
            keep_past=True
        )

        # ============================
        # MEASURE CPU & MEMORY USAGE
        # ============================
        print(f'-----Training model----')
        process = psutil.Process()

        # Train the model
        start_time = time.perf_counter()
        model.fit(X_train_selected_df  ,y_train_target  )
        train_time = time.perf_counter() - start_time
        memory_usage_train = process.memory_info().rss / (1024 * 1024)  # MB


        # get feature names
        X_feature_names = X_train_selected_df.columns.tolist()


        print(f'-----predicting on training data----')

        # Predict on train and validation data
        start_time = time.perf_counter()

        y_pred_train_scaled=model.predict(X_train_selected_df)
        
        

        print(f'-----predicting on validation data----')
        
        y_pred_val_scaled=model.predict(X_val_selected_df) 



        # Inverse transform to original scale
        y_pred_train = scaler_y_pend.inverse_transform(y_pred_train_scaled)
        y_pred_val = scaler_y_pend.inverse_transform(y_pred_val_scaled)


        predict_time = time.perf_counter() - start_time
        memory_usage_predict = process.memory_info().rss / (1024 * 1024)  # MB

        # Convert predictions to DataFrames for easier handling
        y_pred_train_df = pd.DataFrame(y_pred_train, columns=output_cols)
        y_pred_val_df = pd.DataFrame(y_pred_val, columns=output_cols)


        # Get true values aligned with dfy_train and dfy_val indexes
        y_true_train =scaler_y_pend.inverse_transform(dfy_train[output_cols].reset_index(drop=True))
        y_true_train_df = pd.DataFrame(y_true_train, columns=output_cols)

        y_true_val = scaler_y_pend.inverse_transform(dfy_val[output_cols].reset_index(drop=True))
        y_true_val_df = pd.DataFrame(y_true_val, columns=output_cols)



        # save model info to dfs
        metrics_row = {
        'model_name': model_name,
        'na': na,
        'nb': nb,
        'nf': nf,
        'xcase': xcase,
        'test case': case,
        }

        perf_row = {
        'model_name': model_name,
        'na': na,
        'nb': nb,
        'nf': nf,
        'degree': degree,
        'xcase': xcase,
        'train_time': train_time,
        'train_memory_MB': memory_usage_train,
        'predict_time': predict_time,
        'predict_memory_MB': memory_usage_predict
            }
        print(f'-----Eavluating model----')
        # Compute metrics
        if np.any(np.isnan(y_pred_train_df)) or np.any(np.isinf(y_pred_train_df)):
            print("[WARN] Train predictions have NaNs or infs.")

        for col in output_cols:
            mse_train = mean_squared_error(y_true_train_df[col], y_pred_train_df[col])
            r2_train = r2_score(y_true_train_df[col], y_pred_train_df[col])

            mse_val = mean_squared_error(y_true_val_df[col], y_pred_val_df[col])
            r2_val = r2_score(y_true_val_df[col], y_pred_val_df[col])

            # Add to metrics row
            metrics_row[f'r2_train_{col}'] = r2_train
            metrics_row[f'mse_train_{col}'] = mse_train
            metrics_row[f'r2_val_{col}'] = r2_val
            metrics_row[f'mse_val_{col}'] = mse_val

        print("metrics_row:", metrics_row)
        print("perf_row:", perf_row)
        # Append the row dictionaries as new rows in the DataFrames
        metrics_df_ARX_pendulum_only_parralel_eta_only  = pd.concat([metrics_df_ARX_pendulum_only_parralel_eta_only, pd.DataFrame([metrics_row])], ignore_index=True)
        
        




In [ ]:
file_path = os.path.join(save_folder, "metrics_df_ARX_pendulum_only_parralel_eta_only.csv")

# Save the DataFrame
metrics_df_ARX_pendulum_only_parralel_eta_only.to_csv(file_path, index=False)

print(f"Saved metrics_df_ARX_pendulum_only_parralel_eta_only to: {file_path}")


In [ ]:

target_values = ['pendulum']
df_plot = metrics_df_ARX_pendulum_only_parralel_eta_only.copy()
na_values = sorted(df_plot['na'].unique())

for target_var in target_values:
    for na_val in na_values:
        df_na = df_plot[df_plot['na'] == na_val].copy()

        unique_nf = sorted(df_na['nf'].unique())
        unique_nb = sorted(df_na['nb'].unique())

        # Map nf → color using Plotly palette
        color_map = {nf: px.colors.qualitative.Plotly[i % len(px.colors.qualitative.Plotly)]
                     for i, nf in enumerate(unique_nf)}

        fig = make_subplots(
            rows=1, cols=2,
            subplot_titles=[f"R² Train (na={na_val})", f"R² Val (na={na_val})"],
            shared_yaxes=False
        )

        for nf_val in unique_nf:
            df_nf = df_na[df_na['nf'] == nf_val].copy().sort_values(by='nb')
            color = color_map[nf_val]
            nd_label = f"nd={-nf_val}"

            # Train plot
            fig.add_trace(
                go.Scatter(
                    x=df_nf['nb'],
                    y=df_nf[f'r2_train_{target_var}'],
                    mode='lines+markers',
                    line=dict(color=color, width=2),
                    marker=dict(size=8, symbol='circle'),
                    name=nd_label,
                    legendgroup=nd_label
                ),
                row=1, col=1
            )

            # Val plot
            fig.add_trace(
                go.Scatter(
                    x=df_nf['nb'],
                    y=df_nf[f'r2_val_{target_var}'],
                    mode='lines+markers',
                    line=dict(color=color, width=2),
                    marker=dict(size=8, symbol='circle'),
                    name=nd_label,
                    legendgroup=nd_label,
                    showlegend=False
                ),
                row=1, col=2
            )
        #η, η̇  and η̈ 
        fig.update_layout(
            title=dict(
            text = rf"R² Performance of Per-DOF Models Using η only as an Input Feature<br>({target_var})",
            x=0.5,
            xanchor='center'
                        ),
            xaxis_title="nb (Input Lags)",
            xaxis2_title="nb (Input Lags)",
            yaxis_title="R² Score",
            template="plotly_white",
            height=450,
            width=950,
            legend_title="nd"
        )
        #fig.update_yaxes(range=[0.9998, 1], row=1, col=1)
        #fig.update_yaxes(range=[-100, 10], row=1, col=2)

        output_dir = "Annex/figures/ARX_pendulum_eta_only"
        os.makedirs(output_dir, exist_ok=True)

        # Create a clean filename from target and na
        safe_target = target_var.replace(" ", "_")
        filename = f"R2_eta_only_na{na_val}_target_{safe_target}_ver2.png"
        save_path = os.path.join(output_dir, filename)

        # Save the figure
        fig.write_image(save_path, scale=2)  # high-res PNG


        fig.show()


# 1 shared model for all DoFs

In [ ]:
# Base save folder
save_folder = 'ARX/model_output'

# Create the directory if it doesn't exist
os.makedirs(save_folder, exist_ok=True)


In [ ]:
nb_max=[0,1,2,3,4,5,6,7]
nf_max=[0,1,2,3,4,5,6,7]
degree=1
na=2
case='Tp6p8s_Hs2m'
output_cols = ['heave','pitch','pendulum'] 
input_cols=['eta' , 'eta_velocity', 'eta_acceleration']
xcase='eta'

In [ ]:
metrics_3dof_arx_series_eta_only = pd.DataFrame()
perf_3dof_arx_series_eta_only=pd.DataFrame()


In [ ]:
for nb in nb_max:
    for nf in nf_max:
        
         ##
        # Model name
        model_name = F'ARX_3dof_' + xcase + '_na' + str(na) + '_nb' + str(nb) + '_nf' + str(nf)
        # Create the pipeline model
        model = Pipeline([
                ('polynomial', PolynomialFeatures(degree=degree, interaction_only=False)),
                ('linear', LinearRegression())
            ])
            
        # prepare training data 
        print(f'-----Preprocessing case of na={str(na)} ,nb={str(nb)} and nf={str(nf)} , with xcase: {xcase}----')
        # get  df at true scale witAh only this test case
        df_case_train=df_train_full[df_train_full['test_name']==case].reset_index(drop=True)



        dfx_train,dfy_train,yi_train = build_arx_lagged_with_scalers(
                df = df_case_train,
                input_cols  = input_cols,
                output_cols   = output_cols,
                scaler_X_func   = scaler_X_func_nov,
                scaler_y_func   = scaler_y_func,
                na=na,
                nb_past=nb,
                nf_future=nf,
                test_name_col='test_name',
                y_initial_mode='zero'  # 'original' ➔ skip initial rows, 'zero' ➔ pad lags with zeros
        )
            
        y_train_target = dfy_train[output_cols].reset_index(drop=True)



        # prepare validation data
        df_case_val=df_val_full[df_val_full['test_name']==case].reset_index(drop=True)

        dfx_val,dfy_val,yi_val = build_arx_lagged_with_scalers(
                df = df_case_val,
                input_cols  = input_cols,
                output_cols   = output_cols,
                scaler_X_func   = scaler_X_func_nov,
                scaler_y_func   = scaler_y_func ,
                na=na,
                nb_past=nb,
                nf_future=nf,
                test_name_col='test_name',
                y_initial_mode='original'  # 'original' ➔ skip initial rows, 'zero' ➔ pad lags with zeros
        )


        # select eta only in input features
        X_train_selected_df = select_input_features(
            dfx_train.drop(columns='test_name'),
            input_type=xcase,
            keep_future=True,
            keep_past=True
        )

        X_val_selected_df = select_input_features(
            dfx_val.drop(columns='test_name'),
            input_type=xcase,
            keep_future=True,
            keep_past=True
        )

        # ============================
        # MEASURE CPU & MEMORY USAGE
        # ============================
        print(f'-----Training model----')
        process = psutil.Process()

        # Train the model
        start_time = time.perf_counter()
        model.fit(X_train_selected_df  ,y_train_target  )
        train_time = time.perf_counter() - start_time
        memory_usage_train = process.memory_info().rss / (1024 * 1024)  # MB


        # get feature names
        X_feature_names = X_train_selected_df.columns.tolist()


        print(f'-----predicting on training data----')

        # Predict on train and validation data
        start_time = time.perf_counter()

        y_pred_train_scaled,x_used_train = predict_recursive_series(
        model=model,
        X_df=X_train_selected_df,
        output_cols=output_cols,
        X_feature_names=X_feature_names,
        na=na
            )

        print(f'-----predicting on validation data----')
        
        y_pred_val_scaled , x_used_val=predict_recursive_series(
        model=model,
        X_df=X_val_selected_df,
        output_cols=output_cols,
        X_feature_names=X_feature_names,
        na=na
            )

        # Inverse transform to original scale
        y_pred_train = scaler_y.inverse_transform(y_pred_train_scaled)
        y_pred_val = scaler_y.inverse_transform(y_pred_val_scaled)


        predict_time = time.perf_counter() - start_time
        memory_usage_predict = process.memory_info().rss / (1024 * 1024)  # MB

        # Convert predictions to DataFrames for easier handling
        y_pred_train_df = pd.DataFrame(y_pred_train, columns=output_cols)
        y_pred_val_df = pd.DataFrame(y_pred_val, columns=output_cols)


        # Get true values aligned with dfy_train and dfy_val indexes
        y_true_train =scaler_y.inverse_transform(dfy_train[output_cols].reset_index(drop=True))
        y_true_train_df = pd.DataFrame(y_true_train, columns=output_cols)

        y_true_val = scaler_y.inverse_transform(dfy_val[output_cols].reset_index(drop=True))
        y_true_val_df = pd.DataFrame(y_true_val, columns=output_cols)



        # save model info to dfs
        metrics_row = {
        'model_name': model_name,
        'na': na,
        'nb': nb,
        'nf': nf,
        'xcase': xcase,
        'test case': case,
        }

        perf_row = {
        'model_name': model_name,
        'na': na,
        'nb': nb,
        'nf': nf,
        'degree': degree,
        'xcase': xcase,
        'train_time': train_time,
        'train_memory_MB': memory_usage_train,
        'predict_time': predict_time,
        'predict_memory_MB': memory_usage_predict
            }
        print(f'-----Eavluating model----')
        # Compute metrics
        if np.any(np.isnan(y_pred_train_df)) or np.any(np.isinf(y_pred_train_df)):
            print("[WARN] Train predictions have NaNs or infs.")

        for col in output_cols:
            mse_train = mean_squared_error(y_true_train_df[col], y_pred_train_df[col])
            r2_train = r2_score(y_true_train_df[col], y_pred_train_df[col])

            mse_val = mean_squared_error(y_true_val_df[col], y_pred_val_df[col])
            r2_val = r2_score(y_true_val_df[col], y_pred_val_df[col])

            # Add to metrics row
            metrics_row[f'r2_train_{col}'] = r2_train
            metrics_row[f'mse_train_{col}'] = mse_train
            metrics_row[f'r2_val_{col}'] = r2_val
            metrics_row[f'mse_val_{col}'] = mse_val

        print("metrics_row:", metrics_row)
        print("perf_row:", perf_row)
        # Append the row dictionaries as new rows in the DataFrames
        metrics_3dof_arx_series_eta_only  = pd.concat([metrics_3dof_arx_series_eta_only, pd.DataFrame([metrics_row])], ignore_index=True)
        perf_3dof_arx_series_eta_only = pd.concat([perf_3dof_arx_series_eta_only, pd.DataFrame([perf_row])], ignore_index=True)
       
        print(f'-----Saving model----')
        # Save the trained model
        model_save_name = f"{model_name}.joblib"  # You already have model_name variable!
        model_save_path = os.path.join(save_folder, model_save_name)

        # Save with joblib
        joblib.dump(model, model_save_path) 





In [ ]:
# save results 
# Define your new folder path
save_folder = "ARX/metrics_outpus"

# Create the folder if it doesn't exist
os.makedirs(save_folder, exist_ok=True)
# Define the file name
file_path = os.path.join(save_folder, "metrics_3dof_arx_series_eta_only.csv")

# Save the DataFrame
metrics_3dof_arx_series_eta_only.to_csv(file_path, index=False)

print(f"Saved metrics_3dof_arx_series_eta_only to: {file_path}")

# Define the file name
file_path = os.path.join(save_folder, "perf_3dof_arx_series_eta_only.csv")

# Save the DataFrame
perf_3dof_arx_series_eta_only.to_csv(file_path, index=False)

print(f"Saved perf_3dof_arx_series_eta_only to: {file_path}")





## Comparing parrale configration vs series configration predictions

In [ ]:
scaler_y_3dof = MinMaxScaler(feature_range=(-1, 1))

# Fit on training data (DataFrames, not NumPy arrays)

scaler_y_3dof.fit(df_train_full[output_cols])

# define scalling function
# Define the scaling functions (they apply the .transform)

scaler_y_func_3dpf = lambda df: scaler_y_3dof.transform(df)

In [ ]:
na  = 2
nb=2
nf=5
degree = 1
xcase = 'eta'  # xcases
input_cols=['eta' , 'eta_velocity', 'eta_acceleration']
output_cols=['heave', 'pitch',  'pendulum']



In [ ]:

case='Tp6p8s_Hs2m'
model_name = F'ARX_' + xcase + '_na' + str(na) + '_nb' + str(nb) + '_nf' + str(nf)


# Create the pipeline model
model = Pipeline([
        ('polynomial', PolynomialFeatures(degree=1, interaction_only=False)),
        ('linear', LinearRegression())
    ])
    
# prepare training data 
print(f'-----Preprocessing case of na={str(na)} ,nb={str(nb)} and nf={str(nf)} , with xcase: {xcase}----')
# get  df at true scale witAh only this test case
df_case_train=df_train_full[df_train_full['test_name']==case].reset_index(drop=True)



dfx_train,dfy_train,yi_train = build_arx_lagged_with_scalers(
        df = df_case_train,
        input_cols  = input_cols,
        output_cols   = output_cols,
        scaler_X_func   = scaler_X_func,
        scaler_y_func   = scaler_y_func_3dpf,
        na=na,
        nb_past=nb,
        nf_future=nf,
        test_name_col='test_name',
        y_initial_mode='zero'  # 'original' ➔ skip initial rows, 'zero' ➔ pad lags with zeros
)
    
y_train_target = dfy_train[output_cols].reset_index(drop=True)


# select eta only in input features
X_train_selected_df = select_input_features(
    dfx_train.drop(columns='test_name'),
    input_type=xcase,
    keep_future=True,
    keep_past=True
)

model.fit(X_train_selected_df  ,y_train_target  )

print(f'-----predicting on training data----')

# get feature names
X_feature_names = X_train_selected_df.columns.tolist()

print(f'-----predicting on training and validation data----')
# Predict on train and validation data
y_pred_parralel_scaled = model.predict(X_train_selected_df)


In [ ]:

y_pred_series_scaled,x_used_train = predict_recursive_series(
            model=model,
            X_df=X_train_selected_df.iloc[:10000],
            output_cols=output_cols,
            X_feature_names=X_feature_names,
            na=na
                )



In [ ]:

metrics_row={}
# Inverse transform to original scale
y_pred_parralel = scaler_y_3dof.inverse_transform(y_pred_parralel_scaled)
y_pred_series = scaler_y_3dof.inverse_transform(y_pred_series_scaled)



# Convert predictions to DataFrames for easier handling
y_pred_parralel_df = pd.DataFrame(y_pred_parralel, columns=output_cols)
y_pred_series_df = pd.DataFrame(y_pred_series, columns=output_cols)


# Get true values aligned with dfy_train and dfy_val indexes
y_true_train =scaler_y_3dof.inverse_transform(dfy_train[output_cols].reset_index(drop=True))
y_true_train_df = pd.DataFrame(y_true_train, columns=output_cols)





for col in output_cols:
    mse_parralel = mean_squared_error(y_true_train_df[col], y_pred_parralel_df[col])
    r2_parralel = r2_score(y_true_train_df[col], y_pred_parralel_df[col])

    mse_series = mean_squared_error(y_true_train_df[col].iloc[:10000], y_pred_series_df[col])
    r2_series = r2_score(y_true_train_df[col].iloc[:10000], y_pred_series_df[col])

    # Add to metrics row
    metrics_row[f'r2_parralel_{col}'] = r2_parralel
    metrics_row[f'mse_parralel_{col}'] = mse_parralel
    metrics_row[f'r2_series_{col}'] = r2_series
    metrics_row[f'mse_series_{col}'] = mse_series

print("metrics_row:", metrics_row)


In [ ]:
X_train_selected_df

In [ ]:
n_points = 10000
time_steps = np.arange(n_points)  # time steps: 0, 1, ..., 9999

for col in output_cols:
    if 'heave' in col.lower():
        y_label = f"{col.capitalize()} [m]"
     
         
    elif 'pitch' in col.lower() or 'pendulum' in col.lower():
        y_label = f"{col.capitalize()} [°]"
    else:
        y_label = col.capitalize()
        
    fig = go.Figure()

    # True signal (solid blue)
    fig.add_trace(go.Scatter(
        x=time_steps,
        y=y_true_train_df[col].iloc[:n_points],
        mode='lines',
        name='True (Train)',
        line=dict(color='blue', dash='solid')
    ))

    # Series prediction (dashed red)
    fig.add_trace(go.Scatter(
        x=time_steps,
        y=y_pred_series_df[col].iloc[:n_points],
        mode='lines',
        name='Predicted (Series)',
        line=dict(color='black', dash='dash')
    ))

    # Parallel prediction (dotted black)
    fig.add_trace(go.Scatter(
        x=time_steps,
        y=y_pred_parralel_df[col].iloc[:n_points],
        mode='lines',
        name='Predicted (Parallel)',
        line=dict(color='red', dash='dot')
    ))

    fig.update_layout(
        title=f"Train Predictions for {col} - na={na}, nb={nb}, nd={-nf}",
        xaxis_title="Time Step",
        yaxis_title=y_label,
        template="plotly_white",
        width=1000,
        height=400,
        yaxis=dict(range=[-3, 3]),

        legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='right', x=1)
    )

    fig.show()


In [ ]:
# save results 
# Define your new folder path
save_folder = "ARX/metrics_outpus"

# Create the folder if it doesn't exist
os.makedirs(save_folder, exist_ok=True)
# Define the file name
file_path = os.path.join(save_folder, "metrics_df_ARX_parallel.csv")

# Save the DataFrame
metrics_df_ARX_parallel.to_csv(file_path, index=False)

print(f"Saved metrics_df to: {file_path}")

# Define the file name
file_path = os.path.join(save_folder, "perf_df_ARX_parallel.csv")

# Save the DataFrame
perf_df_ARX_parallel.to_csv(file_path, index=False)

print(f"Saved perf_df_ARX_parallel to: {file_path}")

#  1 Model For Heave Only

## Finding Optimal Lags

In [ ]:
# Initialize empty DataFrames before the loop
metrics_df_ARX_heave_only_na2 = pd.DataFrame()
perf_df_ARX_series_heave_only_na2 = pd.DataFrame()

metrics_df_ARX_heave_only  =pd.DataFrame()


In [ ]:
import os
# Base save folder
save_folder = 'ARX/saved_models'

# Create the directory if it doesn't exist
os.makedirs(save_folder, exist_ok=True)



In [ ]:
# define loop values 
na_max  = 2
nb_max= [2]
nf_max=[7]
degree = 1
xcase = 'eta'  # xcases
input_cols=['eta' , 'eta_velocity', 'eta_acceleration']
output_cols = ['heave']



In [ ]:
case='Tp6p8s_Hs2m'
output_cols = ['heave']
#for na in range(4,na_max+1):
na=2


for nb in nb_max:
    for nf in nf_max:
        

        ##
        # Model parameters
        model_name = F'ARX_heave_' + xcase + '_na' + str(na) + '_nb' + str(nb) + '_nf' + str(nf)


        # Create the pipeline model
        model = Pipeline([
                ('polynomial', PolynomialFeatures(degree=degree, interaction_only=False)),
                ('linear', LinearRegression())
            ])
            
        # prepare training data 
        print(f'-----Preprocessing case of na={str(na)} ,nb={str(nb)} and nf={str(nf)} , with xcase: {xcase}----')
        # get  df at true scale witAh only this test case
        df_case_train=df_train_full[df_train_full['test_name']==case].reset_index(drop=True)



        dfx_train,dfy_train,yi_train = build_arx_lagged_with_scalers(
                df = df_case_train,
                input_cols  = input_cols,
                output_cols   = output_cols,
                scaler_X_func   = scaler_X_func_nov,
                scaler_y_func   = scaler_y_func_heave,
                na=na,
                nb_past=nb,
                nf_future=nf,
                test_name_col='test_name',
                y_initial_mode='zero'  # 'original' ➔ skip initial rows, 'zero' ➔ pad lags with zeros
        )
            
        y_train_target = dfy_train[output_cols].reset_index(drop=True)



        # prepare validation data
        df_case_val=df_val_full[df_val_full['test_name']==case].reset_index(drop=True)

        dfx_val,dfy_val,yi_val = build_arx_lagged_with_scalers(
                df = df_case_val,
                input_cols  = input_cols,
                output_cols   = output_cols,
                scaler_X_func   = scaler_X_func_nov,
                scaler_y_func   = scaler_y_func_heave,
                na=na,
                nb_past=nb,
                nf_future=nf,
                test_name_col='test_name',
                y_initial_mode='original'  # 'original' ➔ skip initial rows, 'zero' ➔ pad lags with zeros
        )


        # select eta only in input features
        X_train_selected_df = select_input_features(
            dfx_train.drop(columns='test_name'),
            input_type=xcase,
            keep_future=True,
            keep_past=True
        )

        X_val_selected_df = select_input_features(
            dfx_val.drop(columns='test_name'),
            input_type=xcase,
            keep_future=True,
            keep_past=True
        )

        # ============================
        # MEASURE CPU & MEMORY USAGE
        # ============================
        print(f'-----Training model----')
        process = psutil.Process()

        # Train the model
        start_time = time.perf_counter()
        model.fit(X_train_selected_df  ,y_train_target  )
        train_time = time.perf_counter() - start_time
        memory_usage_train = process.memory_info().rss / (1024 * 1024)  # MB


        # get feature names
        X_feature_names = X_train_selected_df.columns.tolist()


        print(f'-----predicting on training data----')

        # Predict on train and validation data
        start_time = time.perf_counter()

        y_pred_train_scaled,x_used_train = predict_recursive_series(
        model=model,
        X_df=X_train_selected_df,
        output_cols=output_cols,
        X_feature_names=X_feature_names,
        na=na
            )

        print(f'-----predicting on validation data----')
        
        y_pred_val_scaled , x_used_val=predict_recursive_series(
        model=model,
        X_df=X_val_selected_df,
        output_cols=output_cols,
        X_feature_names=X_feature_names,
        na=na
            )



        # Inverse transform to original scale
        y_pred_train = scaler_y_heave.inverse_transform(y_pred_train_scaled)
        y_pred_val = scaler_y_heave.inverse_transform(y_pred_val_scaled)


        predict_time = time.perf_counter() - start_time
        memory_usage_predict = process.memory_info().rss / (1024 * 1024)  # MB

        # Convert predictions to DataFrames for easier handling
        y_pred_train_df = pd.DataFrame(y_pred_train, columns=output_cols)
        y_pred_val_df = pd.DataFrame(y_pred_val, columns=output_cols)


        # Get true values aligned with dfy_train and dfy_val indexes
        y_true_train =scaler_y_heave.inverse_transform(dfy_train[output_cols].reset_index(drop=True))
        y_true_train_df = pd.DataFrame(y_true_train, columns=output_cols)

        y_true_val = scaler_y_heave.inverse_transform(dfy_val[output_cols].reset_index(drop=True))
        y_true_val_df = pd.DataFrame(y_true_val, columns=output_cols)



        # save model info to dfs
        metrics_row = {
        'model_name': model_name,
        'na': na,
        'nb': nb,
        'nf': nf,
        'xcase': xcase,
        'test case': case,
        }

        perf_row = {
        'model_name': model_name,
        'na': na,
        'nb': nb,
        'nf': nf,
        'degree': degree,
        'xcase': xcase,
        'train_time': train_time,
        'train_memory_MB': memory_usage_train,
        'predict_time': predict_time,
        'predict_memory_MB': memory_usage_predict
            }
        print(f'-----Eavluating model----')
        # Compute metrics
        if np.any(np.isnan(y_pred_train_df)) or np.any(np.isinf(y_pred_train_df)):
            print("[WARN] Train predictions have NaNs or infs.")

        for col in output_cols:
            mse_train = mean_squared_error(y_true_train_df[col], y_pred_train_df[col])
            r2_train = r2_score(y_true_train_df[col], y_pred_train_df[col])

            mse_val = mean_squared_error(y_true_val_df[col], y_pred_val_df[col])
            r2_val = r2_score(y_true_val_df[col], y_pred_val_df[col])

            # Add to metrics row
            metrics_row[f'r2_train_{col}'] = r2_train
            metrics_row[f'mse_train_{col}'] = mse_train
            metrics_row[f'r2_val_{col}'] = r2_val
            metrics_row[f'mse_val_{col}'] = mse_val

        print("metrics_row:", metrics_row)
        print("perf_row:", perf_row)
        # Append the row dictionaries as new rows in the DataFrames
        #metrics_df_ARX_heave_only  = pd.concat([metrics_df_ARX_heave_only, pd.DataFrame([metrics_row])], ignore_index=True)
        #perf_df_ARX_series_heave_only_na2 = pd.concat([perf_df_ARX_series_heave_only_na2, pd.DataFrame([perf_row])], ignore_index=True)
        print(f'-----Saving model----')
        # Save the trained model
        model_save_name = f"{model_name}.joblib"  # You already have model_name variable!
        model_save_path = os.path.join(save_folder, model_save_name)

        # Save with joblib
        joblib.dump(model, model_save_path) 




In [ ]:
# save results 
# Define your new folder path
save_folder = "ARX/metrics_outpus"

# Create the folder if it doesn't exist
os.makedirs(save_folder, exist_ok=True)
# Define the file name
file_path = os.path.join(save_folder, "metrics_df_ARX_heave_only.csv")

# Save the DataFrame
metrics_df_ARX_heave_only.to_csv(file_path, index=False)

print(f"Saved metrics_df_ARX_heave_only to: {file_path}")





In [ ]:
#load the saved metrics
metrics_df_ARX_heave_only = pd.read_csv("ARX\metrics_outpus\heave\metrics_df_ARX_heave_only_final.csv")

In [ ]:

# Set target variable and work with the heave DataFrame
target_var = 'heave'
df_plot = metrics_df_ARX_heave_only[metrics_df_ARX_heave_only['na'] == 2 ].copy()

# Loop over each unique na value
na_values = sorted(df_plot['na'].unique())

for na_val in na_values:
    df_na = df_plot[df_plot['na'] == na_val].copy()
    
    # Unique nb values and create a color map for them
    unique_nb = sorted(df_na['nb'].unique())
    color_map = {nb: px.colors.qualitative.Plotly[i % len(px.colors.qualitative.Plotly)]
                 for i, nb in enumerate(unique_nb)}
    
    # Create a figure with two subplots (Train and Validation), sharing the y-axis
    fig = make_subplots(
        rows=1, cols=2,
        subplot_titles=[f"R² Train (na={na_val})", f"R² Val (na={na_val})"],
        shared_yaxes=False
    )
    
    # Loop through each unique nb, plotting curves vs nf (keep x values as nf)
    for nb_val in unique_nb:
        df_nb = df_na[df_na['nb'] == nb_val].copy().sort_values(by='nf')
        color = color_map[nb_val]
        
        # Check if nb is the one to highlight
        if nb_val == 2:
            line_width = 4    # Thicker line for highlighting
            marker_size = 12  # Larger markers for highlighting
            marker_symbol = 'star'
        else:
            line_width = 2
            marker_size = 8
            marker_symbol = 'circle'
            
       # Add train R² trace with original nf as x values
        fig.add_trace(
            go.Scatter(
                x=df_nb['nf'],
                y=df_nb[f'r2_train_{target_var}'],
                mode='lines+markers',
                marker=dict(size=marker_size, symbol=marker_symbol),
                line=dict(color=color, width=line_width),
                name=f'nb={nb_val}',
                legendgroup=f'nb={nb_val}'
            ),
            row=1, col=1
        )
        
        # Add validation R² trace with original nf as x values
        fig.add_trace(
            go.Scatter(
                x=df_nb['nf'],
                y=df_nb[f'r2_val_{target_var}'],
                mode='lines+markers',
                marker=dict(size=marker_size, symbol=marker_symbol),
                line=dict(color=color, width=line_width),
                name=f'nb={nb_val}',
                legendgroup=f'nb={nb_val}',
                showlegend=False,  # legend only displayed in the first subplot
            ),
            row=1, col=2
        )
    
    # Create custom tick labels: positions are the original nf values,
    # but labels show as -nf (e.g. nf=5 -> "-5")
    unique_nf = sorted(df_na['nf'].unique())
    tick_vals = unique_nf  # positions remain as nf values
    tick_text = [f"-{nf}" for nf in unique_nf]  # labels are negative versions

    # Update x-axes for both subplots with custom ticks
    fig.update_xaxes(tickmode='array', tickvals=tick_vals, ticktext=tick_text, row=1, col=1)
    fig.update_xaxes(tickmode='array', tickvals=tick_vals, ticktext=tick_text, row=1, col=2)
    
    # Update overall figure layout with titles and axis labels
    fig.update_layout(
        title_text=f" R² performance of Heave ARX Model for na = {na_val} ",
        xaxis_title="nd",
        xaxis2_title="nd ",
        yaxis_title="R² Score",
        template="plotly_white",
        height=400,
        width=900,
        legend_title="nb",
        yaxis1=dict(range=[0.7, 1.05]),
        yaxis2=dict(range=[0.7, 1.05])
    )
    
    fig.show()


Best Model: na=2 , nb=2 , nf= 7 (case:'Tp6p8s_Hs2m')

## dt senstivety

Series

In [ ]:
metrics_df_ARX_heave_dt_senstivery_test=pd.DataFrame()


In [ ]:
steps=[1,2,3,4,5,6]
case='Tp6p8s_Hs2m'

output_cols = ['heave']
#for na in range(4,na_max+1):
na=2
nb=2
nf=7
degree=1
xcase = 'eta'  # xcases
input_cols=['eta' , 'eta_velocity', 'eta_acceleration']

In [ ]:
# Prepare training data

dfx_train,dfy_train_full,yi_train = build_arx_lagged_with_scalers(
        df = df_case_train,
        input_cols  = input_cols,
        output_cols   = output_cols,
        scaler_X_func   = scaler_X_func_nov,
        scaler_y_func   = scaler_y_func_heave,
        na=na,
        nb_past=nb,
        nf_future=nf,
        test_name_col='test_name',
        y_initial_mode='zero'  # 'original' ➔ skip initial rows, 'zero' ➔ pad lags with zeros
)
    
y_train_target_full = dfy_train_full[output_cols].reset_index(drop=True)



# prepare testing data

dfx_test,dfy_test_full,yi_test = build_arx_lagged_with_scalers(
        df = df_case_test,
        input_cols  = input_cols,
        output_cols   = output_cols,
        scaler_X_func   = scaler_X_func_nov,
        scaler_y_func   = scaler_y_func_heave,
        na=na,
        nb_past=nb,
        nf_future=nf,
        test_name_col='test_name',
        y_initial_mode='original'  # 'original' ➔ skip initial rows, 'zero' ➔ pad lags with zeros
)


# select eta only in input features
X_train_selected_df_full = select_input_features(
    dfx_train.drop(columns='test_name'),
    input_type=xcase,
    keep_future=True,
    keep_past=True
)

X_test_selected_df_full = select_input_features(
    dfx_test.drop(columns='test_name'),
    input_type=xcase,
    keep_future=True,
    keep_past=True
)
    
for step in steps:
    
    dt=0.05*step
    
   
    # resample data
    
    X_train_selected_df= X_train_selected_df_full.iloc[::step].reset_index(drop=True)
    y_train_target= y_train_target_full.iloc[::step].reset_index(drop=True)
    X_test_selected_df= X_test_selected_df_full.iloc[::step].reset_index(drop=True)
    dfy_train= dfy_train_full.iloc[::step].reset_index(drop=True)
    dfy_test= dfy_test_full.iloc[::step].reset_index(drop=True)
    
    
    # Create the pipeline model
    model = Pipeline([
            ('polynomial', PolynomialFeatures(degree=degree, interaction_only=False)),
            ('linear', LinearRegression())
        ])
        
    # prepare training data 
    print(f'-----Preprocessing case of dt {dt} ----')
    # ============================
    # MEASURE CPU & MEMORY USAGE
    # ============================
    print(f'-----Training model----')
    
    model.fit(X_train_selected_df  ,y_train_target )
    


    # get feature names
    X_feature_names = X_train_selected_df.columns.tolist()


    print(f'-----predicting on training data----')

    # Predict on train and testing data
    start_time = time.perf_counter()

    y_pred_train_scaled,x_used_train = predict_recursive_series(
    model=model,
    X_df=X_train_selected_df,
    output_cols=output_cols,
    X_feature_names=X_feature_names,
    na=na
        )

    print(f'-----predicting on testing data----')
    
    y_pred_test_scaled , x_used_test=predict_recursive_series(
    model=model,
    X_df=X_test_selected_df,
    output_cols=output_cols,
    X_feature_names=X_feature_names,
    na=na
        )



    # Inverse transform to original scale
    y_pred_train = scaler_y_heave.inverse_transform(y_pred_train_scaled)
    y_pred_test = scaler_y_heave.inverse_transform(y_pred_test_scaled)


   

    # Convert predictions to DataFrames for easier handling
    y_pred_train_df = pd.DataFrame(y_pred_train, columns=output_cols)
    y_pred_test_df = pd.DataFrame(y_pred_test, columns=output_cols)


    # Get true values aligned with dfy_train and dfy_test indexes
    y_true_train =scaler_y_heave.inverse_transform(dfy_train[output_cols].reset_index(drop=True))
    y_true_train_df = pd.DataFrame(y_true_train, columns=output_cols)

    y_true_test = scaler_y_heave.inverse_transform(dfy_test[output_cols].reset_index(drop=True))
    y_true_test_df = pd.DataFrame(y_true_test, columns=output_cols)



    # save model info to dfs
    metrics_row = {
    
    'na': na,
    'nb': nb,
    'nf': nf,
    'xcase': xcase,
    'test case': case,
    'dt' : dt
    }

    
    print(f'-----Eavluating model----')
    # Compute metrics
    if np.any(np.isnan(y_pred_train_df)) or np.any(np.isinf(y_pred_train_df)):
        print("[WARN] Train predictions have NaNs or infs.")

    for col in output_cols:
        mse_train = mean_squared_error(y_true_train_df[col], y_pred_train_df[col])
        r2_train = r2_score(y_true_train_df[col], y_pred_train_df[col])

        mse_test = mean_squared_error(y_true_test_df[col], y_pred_test_df[col])
        r2_test = r2_score(y_true_test_df[col], y_pred_test_df[col])

        # Add to metrics row
        metrics_row[f'r2_train_{col}'] = r2_train
        metrics_row[f'mse_train_{col}'] = mse_train
        metrics_row[f'r2_test_{col}'] = r2_test
        metrics_row[f'mse_test_{col}'] = mse_test

    print("metrics_row:", metrics_row)
    # Append the row dictionaries as new rows in the DataFrames
    # Make sure metrics_row is a dict, not a list of dicts
    if isinstance(metrics_row, dict):
        row_df = pd.DataFrame([metrics_row])
    else:
        row_df = pd.DataFrame(metrics_row)

    metrics_df_ARX_heave_dt_senstivery_test = pd.concat([metrics_df_ARX_heave_dt_senstivery_test, row_df], ignore_index=True)

    




In [ ]:
metrics_df_ARX_heave_dt_senstivery_test

In [ ]:
# Base save folder
save_folder = 'Results/dt_tests'

# Create the directory if it doesn't exist
os.makedirs(save_folder, exist_ok=True)
# Define the file name
file_path = os.path.join(save_folder, "metrics_df_ARX_heave_dt_senstivery_test-.csv")

# Save the DataFrame
metrics_df_ARX_heave_dt_senstivery_test.to_csv(file_path, index=False)



Trying parallel Configration

In [ ]:
metrics_df_ARX_heave_dt_senstivery_test_parralel=pd.DataFrame()


In [ ]:
steps=[1,2,3,4,5,6]
case='Tp6p8s_Hs2m'

output_cols = ['heave']
#for na in range(4,na_max+1):
na=2
nb=2
nf=7
degree=1
xcase = 'eta'  # xcases
input_cols=['eta' , 'eta_velocity', 'eta_acceleration']

In [ ]:
# prepare training data

dfx_train,dfy_train_full,yi_train = build_arx_lagged_with_scalers(
        df = df_case_train,
        input_cols  = input_cols,
        output_cols   = output_cols,
        scaler_X_func   = scaler_X_func_nov,
        scaler_y_func   = scaler_y_func_heave,
        na=na,
        nb_past=nb,
        nf_future=nf,
        test_name_col='test_name',
        y_initial_mode='zero'  # 'original' ➔ skip initial rows, 'zero' ➔ pad lags with zeros
)
    
y_train_target_full = dfy_train_full[output_cols].reset_index(drop=True)



# prepare testing data

dfx_test,dfy_test_full,yi_test = build_arx_lagged_with_scalers(
        df = df_case_test,
        input_cols  = input_cols,
        output_cols   = output_cols,
        scaler_X_func   = scaler_X_func_nov,
        scaler_y_func   = scaler_y_func_heave,
        na=na,
        nb_past=nb,
        nf_future=nf,
        test_name_col='test_name',
        y_initial_mode='original'  # 'original' ➔ skip initial rows, 'zero' ➔ pad lags with zeros
)


# select eta only in input features
X_train_selected_df_full = select_input_features(
    dfx_train.drop(columns='test_name'),
    input_type=xcase,
    keep_future=True,
    keep_past=True
)

X_test_selected_df_full = select_input_features(
    dfx_test.drop(columns='test_name'),
    input_type=xcase,
    keep_future=True,
    keep_past=True
)
    
for step in steps:
    
    dt=0.05*step
    
   
    # resample data
    
    X_train_selected_df= X_train_selected_df_full.iloc[::step].reset_index(drop=True)
    y_train_target= y_train_target_full.iloc[::step].reset_index(drop=True)
    X_test_selected_df= X_test_selected_df_full.iloc[::step].reset_index(drop=True)
    dfy_train= dfy_train_full.iloc[::step].reset_index(drop=True)
    dfy_test= dfy_test_full.iloc[::step].reset_index(drop=True)
    
    
    # Create the pipeline model
    model = Pipeline([
            ('polynomial', PolynomialFeatures(degree=degree, interaction_only=False)),
            ('linear', LinearRegression())
        ])
        
    # prepare training data 
    print(f'-----Preprocessing case of dt {dt} ----')
    # ============================
    # MEASURE CPU & MEMORY USAGE
    # ============================
    print(f'-----Training model----')
    
    model.fit(X_train_selected_df  ,y_train_target )
    


    # get feature names
    X_feature_names = X_train_selected_df.columns.tolist()


    print(f'-----predicting on training data----')

    # Predict on train and testing data
    start_time = time.perf_counter()

    y_pred_train_scaled = model.predict(X_train_selected_df)

    print(f'-----predicting on Testing data----')
    
    y_pred_test_scaled=model.predict(X_test_selected_df)



    # Inverse transform to original scale
    y_pred_train = scaler_y_heave.inverse_transform(y_pred_train_scaled)
    y_pred_test = scaler_y_heave.inverse_transform(y_pred_test_scaled)


   

    # Convert predictions to DataFrames for easier handling
    y_pred_train_df = pd.DataFrame(y_pred_train, columns=output_cols)
    y_pred_test_df = pd.DataFrame(y_pred_test, columns=output_cols)


    # Get true values aligned with dfy_train and dfy_val indexes
    y_true_train =scaler_y_heave.inverse_transform(dfy_train[output_cols].reset_index(drop=True))
    y_true_train_df = pd.DataFrame(y_true_train, columns=output_cols)

    y_true_test = scaler_y_heave.inverse_transform(dfy_test[output_cols].reset_index(drop=True))
    y_true_test_df = pd.DataFrame(y_true_test, columns=output_cols)



    # save model info to dfs
    metrics_row = {
    
    'na': na,
    'nb': nb,
    'nf': nf,
    'xcase': xcase,
    'test case': case,
    'dt' : dt
    }

    
    print(f'-----Eavluating model----')
    # Compute metrics
    if np.any(np.isnan(y_pred_train_df)) or np.any(np.isinf(y_pred_train_df)):
        print("[WARN] Train predictions have NaNs or infs.")

    for col in output_cols:
        mse_train = mean_squared_error(y_true_train_df[col], y_pred_train_df[col])
        r2_train = r2_score(y_true_train_df[col], y_pred_train_df[col])

        mse_test = mean_squared_error(y_true_test_df[col], y_pred_test_df[col])
        r2_test = r2_score(y_true_test_df[col], y_pred_test_df[col])

        # Add to metrics row
        metrics_row[f'r2_train_{col}'] = r2_train
        metrics_row[f'mse_train_{col}'] = mse_train
        metrics_row[f'r2_test_{col}'] = r2_test
        metrics_row[f'mse_test_{col}'] = mse_test

    print("metrics_row:", metrics_row)
    # Append the row dictionaries as new rows in the DataFrames
    # Make sure metrics_row is a dict, not a list of dicts
    if isinstance(metrics_row, dict):
        row_df = pd.DataFrame([metrics_row])
    else:
        row_df = pd.DataFrame(metrics_row)

    metrics_df_ARX_heave_dt_senstivery_test_parralel = pd.concat([metrics_df_ARX_heave_dt_senstivery_test_parralel, row_df], ignore_index=True)

    




In [ ]:
metrics_df_ARX_heave_dt_senstivery_test_parralel

In [ ]:
# Base save folder
save_folder = 'Results/dt_tests'

# Create the directory if it doesn't exist
os.makedirs(save_folder, exist_ok=True)
# Define the file name
file_path = os.path.join(save_folder, "metrics_df_ARX_heave_dt_senstivery_test_parralel-.csv")

# Save the DataFrame
metrics_df_ARX_heave_dt_senstivery_test_parralel.to_csv(file_path, index=False)



## Data Senstivity

Parallel

In [ ]:
lenght=np.array([0.002,0.004,0.008,0.01,0.02 ,0.05 , 0.1 , 0.2 , 0.3 , 0.4 , 0.5 , 0.6 , 0.7 , 0.8 , 0.9,1])*len(df_case_train)
lenght=lenght.astype(int)

output_cols = ['heave']
#for na in range(4,na_max+1):
na=2
nb=2
nf=7
degree=1

xcase = 'eta'  # xcases
input_cols=['eta' , 'eta_velocity', 'eta_acceleration']

    

# prepare training data



dfx_train,dfy_train,yi_train = build_arx_lagged_with_scalers(
        df = df_case_train,
        input_cols  = input_cols,
        output_cols   = output_cols,
        scaler_X_func   = scaler_X_func_nov,
        scaler_y_func   = scaler_y_func_heave,
        na=na,
        nb_past=nb,
        nf_future=nf,
        test_name_col='test_name',
        y_initial_mode='zero'  # 'original' ➔ skip initial rows, 'zero' ➔ pad lags with zeros
)
    
y_train_target = dfy_train[output_cols].reset_index(drop=True)



# prepare testing data


dfx_test,dfy_test,yi_test = build_arx_lagged_with_scalers(
        df = df_case_test,
        input_cols  = input_cols,
        output_cols   = output_cols,
        scaler_X_func   = scaler_X_func_nov,
        scaler_y_func   = scaler_y_func_heave,
        na=na,
        nb_past=nb,
        nf_future=nf,
        test_name_col='test_name',
        y_initial_mode='original'  # 'original' ➔ skip initial rows, 'zero' ➔ pad lags with zeros
)


# select eta only in input features
X_train_selected_df = select_input_features(
    dfx_train.drop(columns='test_name'),
    input_type=xcase,
    keep_future=True,
    keep_past=True
)

X_test_selected_df = select_input_features(
    dfx_test.drop(columns='test_name'),
    input_type=xcase,
    keep_future=True,
    keep_past=True
)


In [ ]:
metrics_df_ARX_heave_data_test_parralel=pd.DataFrame()


In [ ]:

for l in lenght:
    
    


    # Create the pipeline model
    model = Pipeline([
            ('polynomial', PolynomialFeatures(degree=degree, interaction_only=False)),
            ('linear', LinearRegression())
        ])
        
    # prepare training data 
    print(f'-----Preprocessing case of lenght {str(l)} ----')
    # ============================
    # MEASURE CPU & MEMORY USAGE
    # ============================
    print(f'-----Training model----')
    
    model.fit(X_train_selected_df[0:l]  ,y_train_target[0:l] )
    


    # get feature names
    X_feature_names = X_train_selected_df.columns.tolist()


    print(f'-----predicting on training data----')

    # Predict on train and testing data
    start_time = time.perf_counter()

    y_pred_train_scaled=model.predict(X_train_selected_df[0:l])

    print(f'-----predicting on Testing data----')
    
    y_pred_test_scaled =model.predict(X_test_selected_df)



    # Inverse transform to original scale
    y_pred_train = scaler_y_heave.inverse_transform(y_pred_train_scaled)
    y_pred_test = scaler_y_heave.inverse_transform(y_pred_test_scaled)


   

    # Convert predictions to DataFrames for easier handling
    y_pred_train_df = pd.DataFrame(y_pred_train, columns=output_cols)
    y_pred_test_df = pd.DataFrame(y_pred_test, columns=output_cols)


    # Get true testues aligned with dfy_train and dfy_test indexes
    y_true_train =scaler_y_heave.inverse_transform(dfy_train[output_cols][0:l].reset_index(drop=True))
    y_true_train_df = pd.DataFrame(y_true_train, columns=output_cols)

    y_true_test = scaler_y_heave.inverse_transform(dfy_test[output_cols].reset_index(drop=True))
    y_true_test_df = pd.DataFrame(y_true_test, columns=output_cols)



    # save model info to dfs
    metrics_row = {
    'model_name': model_name,
    'na': na,
    'nb': nb,
    'nf': nf,
    'xcase': xcase,
    'test case': case,
    'lenght' : l
    }

    
    print(f'-----Eavluating model----')
    # Compute metrics
    if np.any(np.isnan(y_pred_train_df)) or np.any(np.isinf(y_pred_train_df)):
        print("[WARN] Train predictions have NaNs or infs.")

    
    mse_train = mean_squared_error(y_true_train_df['heave'], y_pred_train_df['heave'])
    r2_train = r2_score(y_true_train_df['heave'], y_pred_train_df['heave'])

    mse_test = mean_squared_error(y_true_test_df['heave'], y_pred_test_df['heave'])
    r2_test = r2_score(y_true_test_df['heave'], y_pred_test_df['heave'])

    # Add to metrics row
    metrics_row[f'r2_train_heave'] = r2_train
    metrics_row[f'mse_train_heave'] = mse_train
    metrics_row[f'r2_test_heave'] = r2_test
    metrics_row[f'mse_test_heave'] = mse_test

    print("metrics_row:", metrics_row)
    # Append the row dictionaries as new rows in the DataFrames
    # Make sure metrics_row is a dict, not a list of dicts
    if isinstance(metrics_row, dict):
        row_df = pd.DataFrame([metrics_row])
    else:
        row_df = pd.DataFrame(metrics_row)

    metrics_df_ARX_heave_data_test_parralel = pd.concat([metrics_df_ARX_heave_data_test_parralel, row_df], ignore_index=True)

    
   

In [ ]:
# Base save folder
save_folder = 'Results/data_tests'

# Create the directory if it doesn't exist
os.makedirs(save_folder, exist_ok=True)
# Define the file name
file_path = os.path.join(save_folder, "metrics_df_ARX_heave_data_test_parralel-.csv")

# Save the DataFrame
metrics_df_ARX_heave_data_test_parralel.to_csv(file_path, index=False)



Series

In [ ]:
metrics_df_ARX_heave_data_test_series=pd.DataFrame()

In [ ]:

for l in lenght:
    
    


    # Create the pipeline model
    model = Pipeline([
            ('polynomial', PolynomialFeatures(degree=degree, interaction_only=False)),
            ('linear', LinearRegression())
        ])
        
    # prepare training data 
    print(f'-----Preprocessing case of lenght {str(l)} ----')
    # ============================
    # MEASURE CPU & MEMORY USAGE
    # ============================
    print(f'-----Training model----')
    
    model.fit(X_train_selected_df[0:l]  ,y_train_target[0:l] )
    


    # get feature names
    X_feature_names = X_train_selected_df.columns.tolist()


    print(f'-----predicting on training data----')

    # Predict on train and Testing data
    start_time = time.perf_counter()

    y_pred_train_scaled,x_used_train = predict_recursive_series(
    model=model,
    X_df=X_train_selected_df[0:l],
    output_cols=output_cols,
    X_feature_names=X_feature_names,
    na=na
        )

    print(f'-----predicting on Testing data----')
    
    y_pred_test_scaled , x_used_test=predict_recursive_series(
    model=model,
    X_df=X_test_selected_df,
    output_cols=output_cols,
    X_feature_names=X_feature_names,
    na=na
        )



    # Inverse transform to original scale
    y_pred_train = scaler_y_heave.inverse_transform(y_pred_train_scaled)
    y_pred_test = scaler_y_heave.inverse_transform(y_pred_test_scaled)


   

    # Convert predictions to DataFrames for easier handling
    y_pred_train_df = pd.DataFrame(y_pred_train, columns=output_cols)
    y_pred_test_df = pd.DataFrame(y_pred_test, columns=output_cols)


    # Get true values aligned with dfy_train and dfy_test indexes
    y_true_train =scaler_y_heave.inverse_transform(dfy_train[output_cols][0:l].reset_index(drop=True))
    y_true_train_df = pd.DataFrame(y_true_train, columns=output_cols)

    y_true_test = scaler_y_heave.inverse_transform(dfy_test[output_cols].reset_index(drop=True))
    y_true_test_df = pd.DataFrame(y_true_test, columns=output_cols)



    # save model info to dfs
    metrics_row = {
    'model_name': model_name,
    'na': na,
    'nb': nb,
    'nf': nf,
    'xcase': xcase,
    'test case': case,
    'lenght' : l
    }

    
    print(f'-----Eavluating model----')
    # Compute metrics
    if np.any(np.isnan(y_pred_train_df)) or np.any(np.isinf(y_pred_train_df)):
        print("[WARN] Train predictions have NaNs or infs.")

    
    mse_train = mean_squared_error(y_true_train_df['heave'], y_pred_train_df['heave'])
    r2_train = r2_score(y_true_train_df['heave'], y_pred_train_df['heave'])

    mse_test = mean_squared_error(y_true_test_df['heave'], y_pred_test_df['heave'])
    r2_test = r2_score(y_true_test_df['heave'], y_pred_test_df['heave'])

    # Add to metrics row
    metrics_row[f'r2_train_heave'] = r2_train
    metrics_row[f'mse_train_heave'] = mse_train
    metrics_row[f'r2_test_heave'] = r2_test
    metrics_row[f'mse_test_heave'] = mse_test

    print("metrics_row:", metrics_row)
    # Append the row dictionaries as new rows in the DataFrames
    # Make sure metrics_row is a dict, not a list of dicts
    if isinstance(metrics_row, dict):
        row_df = pd.DataFrame([metrics_row])
    else:
        row_df = pd.DataFrame(metrics_row)

    metrics_df_ARX_heave_data_test_series = pd.concat([metrics_df_ARX_heave_data_test_series, row_df], ignore_index=True)

    
    print(f'-----Saving model----')
    



In [ ]:
# Base save folder
save_folder = 'Results/data_tests'

# Create the directory if it doesn't exist
os.makedirs(save_folder, exist_ok=True)
# Define the file name
file_path = os.path.join(save_folder, "metrics_df_ARX_heave_data_test_series-.csv")

# Save the DataFrame
metrics_df_ARX_heave_data_test_series.to_csv(file_path, index=False)



## Testing

In [ ]:
input_cols=['eta' , 'eta_velocity', 'eta_acceleration']

In [ ]:
df_case_train=df_train_full[df_train_full['test_name']==case].reset_index(drop=True)



dfx_train,dfy_train,yi_train = build_arx_lagged_with_scalers(
        df = df_case_train,
        input_cols  = input_cols,
        output_cols   = output_cols,
        scaler_X_func   = scaler_X_func_nov,
        scaler_y_func   = scaler_y_func_heave,
        na=2,
        nb_past=2,
        nf_future=6,
        test_name_col='test_name',
        y_initial_mode='zero'  # 'original' ➔ skip initial rows, 'zero' ➔ pad lags with zeros
)
    
y_train_target = dfy_train[output_cols].reset_index(drop=True)





# select eta only in input features
X_train_selected_df = select_input_features(
    dfx_train.drop(columns='test_name'),
    input_type=xcase,
    keep_future=True,
    keep_past=True
)


    


# Create the pipeline model
model = Pipeline([
        ('polynomial', PolynomialFeatures(degree=1, interaction_only=False)),
        ('linear', LinearRegression())
    ])
    


model.fit(X_train_selected_df  ,y_train_target )
    

In [ ]:
# Define the best model parameters
best_na = 2
best_nb = 2
best_nf = 7
case = 'Tp6p8s_Hs2m'  # test case
best_degree = 1
best_xcase = 'eta'
best_input_cols = ['eta', 'eta_velocity', 'eta_acceleration']
best_output_cols = ['heave']
# load the best model
folder='ARX/saved_models'
model_name = f"ARX_heave_{best_xcase}_na{best_na}_nb{best_nb}_nf{best_nf}.joblib"
# Load the model
model_path = os.path.join(folder, model_name)  
model = joblib.load(model_path)

# print all test cases
test_cases = df_test_full['test_name'].unique()
print("All test cases:")
print(test_cases)  



In [ ]:
input_cols = ['eta']  # original inputs
output_cols = ['heave']  # outputs

# Construct feature names in the exact order used by the data pipeline
feature_names = []

# 1. Current input signals (no lag)
feature_names += input_cols

# 2. Output lags (autoregressive part)
for lag in range(1, best_na + 1):
    for out_col in output_cols:
        feature_names.append(f"{out_col}_lag_{lag}")

# 3. Past input lags
for lag in range(1, best_nb + 1):
    for in_col in input_cols:
        feature_names.append(f"{in_col}_past_{lag}")

# 4. Future input lags
for lag in range(1, best_nf + 1):
    for in_col in input_cols:
        feature_names.append(f"{in_col}_future_{lag}")

# View the final ordered feature list
print("Feature names used in X_lagged_df:")
print(feature_names)

In [ ]:
poly = model.named_steps['polynomial']
linear = model.named_steps['linear']

# Use the names remembered from training
input_names = poly.get_feature_names_out(poly.feature_names_in_)

# Coefficients and intercept
coefficients = linear.coef_.flatten()
intercept = float(linear.intercept_)


# Build equation
equation_terms = [f"{coef:.4f} * {name}" for coef, name in zip(coefficients, input_names)]
equation = f"{intercept:.4f} + " + " + ".join(equation_terms)

print("\n📘 Model Equation:")
print(f"y = {equation}")


In [ ]:
metrics_df_test_heave_na2_nb2_nf7_val = pd.DataFrame()  # Will collect all test metrics
training_case='Tp6p8s_Hs2m'

In [ ]:
cases=df_test_full['test_name'].unique()
for case in cases:
    # Filter the test data
    df_case_test = df_test_full[df_test_full['test_name'] == case].reset_index(drop=True)
    # Prepare the test data
    dfx_test, dfy_test, yi_test = build_arx_lagged_with_scalers(
        df=df_case_test,
        input_cols=best_input_cols,
        output_cols=best_output_cols,
        scaler_X_func   = scaler_X_func_nov,
        scaler_y_func   = scaler_y_func_heave,
        na=best_na,
        nb_past=best_nb,
        nf_future=best_nf,
        test_name_col='test_name',
        y_initial_mode='original'  # 'original' ➔ skip initial rows, 'zero' ➔ pad lags with zeros
    )

    # Select only the best input features
    X_test_selected_df = select_input_features(
        dfx_test.drop(columns='test_name'),
        input_type=best_xcase,
        keep_future=True,
        keep_past=True
    )

    # get feature names from model
    X_feature_names = X_test_selected_df.columns.tolist()

    # Predict on test data
    y_pred_test_scaled, x_used_test = predict_recursive_series(
        model=model,
        X_df=X_test_selected_df,
        output_cols=best_output_cols,
        X_feature_names=X_feature_names,
        na=best_na
    )

    # Inverse transform to original scale
    y_pred_test = scaler_y_heave.inverse_transform(y_pred_test_scaled)
    # Convert to DataFrame
    y_pred_test_df = pd.DataFrame(y_pred_test, columns=best_output_cols)

    # Get true values aligned with dfy_test indexes
    y_true_test = scaler_y_heave.inverse_transform(dfy_test[best_output_cols].reset_index(drop=True))
    y_true_test_df = pd.DataFrame(y_true_test, columns=best_output_cols)

    # Compute metrics
    metrics_test = {}
    metrics_test[f'test case'] = case
    if case == training_case:
        metrics_test[f'Comments'] = 'case used for training'
   
    mse_test = mean_squared_error(y_true_test_df['heave'], y_pred_test_df['heave'])
    r2_test = r2_score(y_true_test_df['heave'], y_pred_test_df['heave'])

    
    metrics_test[f'r2_test_heave'] = r2_test
    metrics_test[f'mse_test_heave'] = mse_test
    
    print("metrics:", metrics_test)
    # Append the row dictionaries as new rows in the DataFrame
    metrics_df_test_heave_na2_nb2_nf7_val = pd.concat([metrics_df_test_heave_na2_nb2_nf7_val, pd.DataFrame([metrics_test])], ignore_index=True)
        # Save true and predicted DataFrames to CSV files
    y_true_test_df.to_csv(f'Results/ARX/Testing/heave/y_true_test_heave_{case}.csv', index=False)
    y_pred_test_df.to_csv(f'Results/ARX/Testing/heave/y_pred_test_heave_{case}.csv', index=False)

    

In [ ]:
# save results 
# Define your new folder path
save_folder = "ARX/best_model/metrics_outpus"

# Create the folder if it doesn't exist
os.makedirs(save_folder, exist_ok=True)
# Define the file name
file_path = os.path.join(save_folder, "metrics_df_test_heave_na2_nb2_nf7_val.csv")

# Save the DataFrame
metrics_df_test_heave_na2_nb2_nf7_val.to_csv(file_path, index=False)

print(f"Saved metrics_df_test_heave_na2_nb2_nf7_val to: {file_path}")

## Noisy Data Test

In [ ]:
# now get the best model and predict on the test data
import os

# Define the best model parameters
best_na = 2
best_nb = 2
best_nf = 7
case = 'Tp6p8s_Hs2m'  # test case
best_degree = 1
best_xcase = 'eta'
best_input_cols = ['eta', 'eta_velocity', 'eta_acceleration']
best_output_cols = ['heave']
# load the best model
folder='ARX/saved_models'
model_name = f"ARX_heave_{best_xcase}_na{best_na}_nb{best_nb}_nf{best_nf}.joblib"
# Load the model
model_path = os.path.join(folder, model_name)  
model = joblib.load(model_path)

In [ ]:
# load Noisy data
df_case_test_noisy=pd.read_csv('Results/df_case_test_noisy.csv')
df_case_test_noisy

In [ ]:
#prepare the test data
dfx_test, dfy_test, yi_test = build_arx_lagged_with_scalers(
    df=df_case_test_noisy,
    input_cols=best_input_cols,
    output_cols=best_output_cols,
    scaler_X_func   = scaler_X_func_nov,
    scaler_y_func   = scaler_y_func_heave,
    na=best_na,
    nb_past=best_nb,
    nf_future=best_nf,
    test_name_col='test_name',
    y_initial_mode='original'  # 'original' ➔ skip initial rows, 'zero' ➔ pad lags with zeros
)

# Select only the best input features
X_test_selected_df = select_input_features(
    dfx_test.drop(columns='test_name'),
    input_type=best_xcase,
    keep_future=True,
    keep_past=True
)

# get feature names from model
X_feature_names = X_test_selected_df.columns.tolist()

# Predict on test data
y_pred_test_scaled, x_used_test = predict_recursive_series(
    model=model,
    X_df=X_test_selected_df,
    output_cols=best_output_cols,
    X_feature_names=X_feature_names,
    na=best_na
)

# Inverse transform to original scale
y_pred_test = scaler_y_heave.inverse_transform(y_pred_test_scaled)
# Convert to DataFrame
y_pred_test_df = pd.DataFrame(y_pred_test, columns=best_output_cols)

# Get true values aligned with dfy_test indexes
y_true_test = scaler_y_heave.inverse_transform(dfy_test[best_output_cols].reset_index(drop=True))
y_true_test_df = pd.DataFrame(y_true_test, columns=best_output_cols)

# Compute metrics
metrics_test = {}
metrics_test[f'test case'] = case


mse_test = mean_squared_error(y_true_test_df['heave'], y_pred_test_df['heave'])
r2_test = r2_score(y_true_test_df['heave'], y_pred_test_df['heave'])


metrics_test[f'r2_test_heave'] = r2_test
metrics_test[f'mse_test_heave'] = mse_test

print("metrics:", metrics_test)

In [ ]:
y_true_test_df.to_csv(f'Results/noisy test/ARX_heave_true.csv', index=False)
y_pred_test_df.to_csv(f'Results/noisy test/ARX_heave_pred.csv', index=False)

# 1 model for pitch only

## Finding Optimal Lags

In [ ]:
# Initialize empty DataFrames before the loop
metrics_df_ARX_series_pitch_only = pd.DataFrame()
perf_df_ARX_series_pitch_only = pd.DataFrame()


In [ ]:

# Base save folder
save_folder = 'ARX/saved_models'

# Create the directory if it doesn't exist
os.makedirs(save_folder, exist_ok=True)

In [ ]:
# define loop values 
na_max  = [2]
nb_max= [2]
nf_max=[2]
degree = 1
xcase = 'eta'  # xcases
input_cols=['eta' , 'eta_velocity', 'eta_acceleration']
output_cols = ['pitch']

In [ ]:
output_cols = ['pitch']
case='Tp6p8s_Hs2m'
for na in na_max:
    for nb in nb_max:
        for nf in nf_max:
            
    
            ##
            # Model parameters
            model_name = F'ARX_pitch' + xcase + '_na' + str(na) + '_nb' + str(nb) + '_nf' + str(nf)


            # Create the pipeline model
            model = Pipeline([
                    ('polynomial', PolynomialFeatures(degree=degree, interaction_only=False)),
                    ('linear', LinearRegression())
                ])
                
            # prepare training data 
            print(f'-----Preprocessing case of na={str(na)} ,nb={str(nb)} and nf={str(nf)} , with xcase: {xcase}----')
            # get  df at true scale witAh only this test case
            df_case_train=df_train_full[df_train_full['test_name']==case].reset_index(drop=True)



            dfx_train,dfy_train,yi_train = build_arx_lagged_with_scalers(
                    df = df_case_train,
                    input_cols  = input_cols,
                    output_cols   = output_cols,
                    scaler_X_func   = scaler_X_func_nov,
                    scaler_y_func   = scaler_y_func_pitch,
                    na=na,
                    nb_past=nb,
                    nf_future=nf,
                    test_name_col='test_name',
                    y_initial_mode='zero'  # 'original' ➔ skip initial rows, 'zero' ➔ pad lags with zeros
            )
                
            y_train_target = dfy_train[output_cols].reset_index(drop=True)



            # prepare validation data
            df_case_val=df_val_full[df_val_full['test_name']==case].reset_index(drop=True)

            dfx_val,dfy_val,yi_val = build_arx_lagged_with_scalers(
                    df = df_case_val,
                    input_cols  = input_cols,
                    output_cols   = output_cols,
                    scaler_X_func   = scaler_X_func_nov,
                    scaler_y_func   = scaler_y_func_pitch,
                    na=na,
                    nb_past=nb,
                    nf_future=nf,
                    test_name_col='test_name',
                    y_initial_mode='original'  # 'original' ➔ skip initial rows, 'zero' ➔ pad lags with zeros
            )


            # select eta only in input features
            X_train_selected_df = select_input_features(
                dfx_train.drop(columns='test_name'),
                input_type=xcase,
                keep_future=True,
                keep_past=True
            )

            X_val_selected_df = select_input_features(
                dfx_val.drop(columns='test_name'),
                input_type=xcase,
                keep_future=True,
                keep_past=True
            )

            # ============================
            # MEASURE CPU & MEMORY USAGE
            # ============================
            print(f'-----Training model----')
            process = psutil.Process()

            # Train the model
            start_time = time.perf_counter()
            model.fit(X_train_selected_df  ,y_train_target  )
            train_time = time.perf_counter() - start_time
            memory_usage_train = process.memory_info().rss / (1024 * 1024)  # MB


            # get feature names
            X_feature_names = X_train_selected_df.columns.tolist()


            print(f'-----predicting on training data----')

            # Predict on train and validation data
            start_time = time.perf_counter()

            y_pred_train_scaled,x_used_train = predict_recursive_series(
            model=model,
            X_df=X_train_selected_df,
            output_cols=output_cols,
            X_feature_names=X_feature_names,
            na=na
                )

            print(f'-----predicting on validation data----')
            
            y_pred_val_scaled , x_used_val=predict_recursive_series(
            model=model,
            X_df=X_val_selected_df,
            output_cols=output_cols,
            X_feature_names=X_feature_names,
            na=na
                )



            # Inverse transform to original scale
            y_pred_train = scaler_y_pitch.inverse_transform(y_pred_train_scaled)
            y_pred_val = scaler_y_pitch.inverse_transform(y_pred_val_scaled)


            predict_time = time.perf_counter() - start_time
            memory_usage_predict = process.memory_info().rss / (1024 * 1024)  # MB

            # Convert predictions to DataFrames for easier handling
            y_pred_train_df = pd.DataFrame(y_pred_train, columns=output_cols)
            y_pred_val_df = pd.DataFrame(y_pred_val, columns=output_cols)


            # Get true values aligned with dfy_train and dfy_val indexes
            y_true_train =scaler_y_pitch.inverse_transform(dfy_train[output_cols].reset_index(drop=True))
            y_true_train_df = pd.DataFrame(y_true_train, columns=output_cols)

            y_true_val = scaler_y_pitch.inverse_transform(dfy_val[output_cols].reset_index(drop=True))
            y_true_val_df = pd.DataFrame(y_true_val, columns=output_cols)



            # save model info to dfs
            metrics_row = {
            'model_name': model_name,
            'na': na,
            'nb': nb,
            'nf': nf,
            'xcase': xcase,
            'test case': case,
            }

            perf_row = {
            'model_name': model_name,
            'na': na,
            'nb': nb,
            'nf': nf,
            'degree': degree,
            'xcase': xcase,
            'train_time': train_time,
            'train_memory_MB': memory_usage_train,
            'predict_time': predict_time,
            'predict_memory_MB': memory_usage_predict
                }
            print(f'-----Eavluating model----')
            # Compute metrics
            if np.any(np.isnan(y_pred_train_df)) or np.any(np.isinf(y_pred_train_df)):
                print("[WARN] Train predictions have NaNs or infs.")

            for col in output_cols:
                mse_train = mean_squared_error(y_true_train_df[col], y_pred_train_df[col])
                r2_train = r2_score(y_true_train_df[col], y_pred_train_df[col])

                mse_val = mean_squared_error(y_true_val_df[col], y_pred_val_df[col])
                r2_val = r2_score(y_true_val_df[col], y_pred_val_df[col])

                # Add to metrics row
                metrics_row[f'r2_train_{col}'] = r2_train
                metrics_row[f'mse_train_{col}'] = mse_train
                metrics_row[f'r2_val_{col}'] = r2_val
                metrics_row[f'mse_val_{col}'] = mse_val

            print("metrics_row:", metrics_row)
            print("perf_row:", perf_row)
            # Append the row dictionaries as new rows in the DataFrames
            metrics_df_ARX_series_pitch_only  = pd.concat([metrics_df_ARX_series_pitch_only, pd.DataFrame([metrics_row])], ignore_index=True)
            #perf_df_ARX_series_pitch_only = pd.concat([perf_df_ARX_series_pitch_only, pd.DataFrame([perf_row])], ignore_index=True)
            print(f'-----Saving model----')
            # Save the trained model
            model_save_name = f"{model_name}.joblib"  # You already have model_name variable!
            model_save_path = os.path.join(save_folder, model_save_name)

            # Save with joblib
            joblib.dump(model, model_save_path) 




In [ ]:
# save results 
# Define your new folder path
save_folder = "ARX/metrics_outpus"

# Create the folder if it doesn't exist
os.makedirs(save_folder, exist_ok=True)
# Define the file name
file_path = os.path.join(save_folder, "metrics_df_ARX_series_pitch_only.csv")

# Save the DataFrame
metrics_df_ARX_series_pitch_only.to_csv(file_path, index=False)

print(f"Saved metrics_df_ARX_series_pitch_only to: {file_path}")

# Define the file name
file_path = os.path.join(save_folder, "perf_df_ARX_series_pitch_only.csv")

# Save the DataFrame
perf_df_ARX_series_pitch_only.to_csv(file_path, index=False)

print(f"Saved perf_df_ARX_series_pitch_only to: {file_path}")



In [ ]:
# Load the saved metrics DataFrame
metrics_df_ARX_series_pitch_only= pd.read_csv(r"D:\thesis\Work-space\ARX\metrics_outpus\pitch\metrics_df_ARX_series_pitch_only.csv")

In [ ]:

# Set target variable and work with the pitch DataFrame
target_var = 'pitch'
df_plot = metrics_df_ARX_series_pitch_only[metrics_df_ARX_series_pitch_only['na'] == 2].copy()

# Loop over each unique na value (here, only na=2)
na_values = sorted(df_plot['na'].unique())

for na_val in na_values:
    df_na = df_plot[df_plot['na'] == na_val].copy()
    
    # Unique nb values and create a color map for them
    unique_nb = sorted(df_na['nb'].unique())
    color_map = {nb: px.colors.qualitative.Plotly[i % len(px.colors.qualitative.Plotly)]
                 for i, nb in enumerate(unique_nb)}
    
    # Create a figure with two subplots (Train and Validation), sharing the y-axis
    fig = make_subplots(
        rows=1, cols=2,
        subplot_titles=[f"R² Train (na={na_val})", f"R² Val (na={na_val})"],
        shared_yaxes=False
    )
    
    # Loop through each unique nb, plotting curves vs nf (x values remain nf)
    for nb_val in unique_nb:
        df_nb = df_na[df_na['nb'] == nb_val].copy().sort_values(by='nf')
        color = color_map[nb_val]
        
        # Check if nb is the one to highlight
        if nb_val == 2:
            line_width = 4    # Thicker line for highlighting
            marker_size = 12  # Larger markers for highlighting
            marker_symbol = 'star'
        else:
            line_width = 2
            marker_size = 8
            marker_symbol = 'circle'
        
        # Add train R² trace with original nf as x values
        fig.add_trace(
            go.Scatter(
                x=df_nb['nf'],
                y=df_nb[f'r2_train_{target_var}'],
                mode='lines+markers',
                marker=dict(size=marker_size, symbol=marker_symbol),
                line=dict(color=color, width=line_width),
                name=f'nb={nb_val}',
                legendgroup=f'nb={nb_val}'
            ),
            row=1, col=1
        )
        
        # Add validation R² trace with original nf as x values
        fig.add_trace(
            go.Scatter(
                x=df_nb['nf'],
                y=df_nb[f'r2_val_{target_var}'],
                mode='lines+markers',
                marker=dict(size=marker_size, symbol=marker_symbol),
                line=dict(color=color, width=line_width),
                name=f'nb={nb_val}',
                legendgroup=f'nb={nb_val}',
                showlegend=False,  # legend only displayed in the first subplot
            ),
            row=1, col=2
        )
    
    # Create custom tick labels: positions are the original nf values,
    # but labels show as -nf (e.g. nf=5 -> "-5")
    unique_nf = sorted(df_na['nf'].unique())
    tick_vals = unique_nf  # positions remain as nf values
    tick_text = [f"-{nf}" for nf in unique_nf]  # labels are negative versions

    # Update x-axes for both subplots with custom ticks
    fig.update_xaxes(tickmode='array', tickvals=tick_vals, ticktext=tick_text, row=1, col=1)
    fig.update_xaxes(tickmode='array', tickvals=tick_vals, ticktext=tick_text, row=1, col=2)
    
    # Update overall figure layout with titles and axis labels
    fig.update_layout(
        title_text=f"R² performance of Pitch ARX Model for na = {na_val}",
        xaxis_title="nd ",
        xaxis2_title="nd ",
        yaxis_title="R² Score",
        template="plotly_white",
        height=400,
        width=900,
        legend_title="nb ",
        yaxis1=dict(range=[0.6, 1.05]),
        yaxis2=dict(range=[0.6, 1.05]),
    )
    
    fig.show()


Best Model at na 2 , nb 2 , nf 7

## dt Senstivity

In [ ]:
metrics_df_ARX_picth_dt_senstivery_test=pd.DataFrame()


In [ ]:
steps=[1,2,3,4,5,6]
case='Tp6p8s_Hs2m'

output_cols = ['pitch']
#for na in range(4,na_max+1):
na=2
nb=2
nf=7
degree=1
xcase = 'eta'  # xcases
input_cols=['eta' , 'eta_velocity', 'eta_acceleration']

In [ ]:

# prepare training data

dfx_train,dfy_train_full,yi_train = build_arx_lagged_with_scalers(
        df = df_case_train,
        input_cols  = input_cols,
        output_cols   = output_cols,
        scaler_X_func   = scaler_X_func_nov,
        scaler_y_func   = scaler_y_func_pitch,
        na=na,
        nb_past=nb,
        nf_future=nf,
        test_name_col='test_name',
        y_initial_mode='zero'  # 'original' ➔ skip initial rows, 'zero' ➔ pad lags with zeros
)
    
y_train_target_full = dfy_train_full[output_cols].reset_index(drop=True)



# prepare testing data

dfx_test,dfy_test_full,yi_test = build_arx_lagged_with_scalers(
        df = df_case_test,
        input_cols  = input_cols,
        output_cols   = output_cols,
        scaler_X_func   = scaler_X_func_nov,
        scaler_y_func   = scaler_y_func_pitch,
        na=na,
        nb_past=nb,
        nf_future=nf,
        test_name_col='test_name',
        y_initial_mode='original'  # 'original' ➔ skip initial rows, 'zero' ➔ pad lags with zeros
)


# select eta only in input features
X_train_selected_df_full = select_input_features(
    dfx_train.drop(columns='test_name'),
    input_type=xcase,
    keep_future=True,
    keep_past=True
)

X_test_selected_df_full = select_input_features(
    dfx_test.drop(columns='test_name'),
    input_type=xcase,
    keep_future=True,
    keep_past=True
)
    
for step in steps:
    
    dt=0.05*step
    
    
    # resample data
    
    X_train_selected_df= X_train_selected_df_full.iloc[::step].reset_index(drop=True)
    y_train_target= y_train_target_full.iloc[::step].reset_index(drop=True)
    X_test_selected_df= X_test_selected_df_full.iloc[::step].reset_index(drop=True)
    dfy_train= dfy_train_full.iloc[::step].reset_index(drop=True)
    dfy_test= dfy_test_full.iloc[::step].reset_index(drop=True)
    
    
    # Create the pipeline model
    model = Pipeline([
            ('polynomial', PolynomialFeatures(degree=degree, interaction_only=False)),
            ('linear', LinearRegression())
        ])
        
    # prepare training data 
    print(f'-----Preprocessing case of dt {dt} ----')
    # ============================
    # MEASURE CPU & MEMORY USAGE
    # ============================
    print(f'-----Training model----')
    
    model.fit(X_train_selected_df  ,y_train_target )
    


    # get feature names
    X_feature_names = X_train_selected_df.columns.tolist()


    print(f'-----predicting on training data----')

    # Predict on train and Testing data
    start_time = time.perf_counter()

    y_pred_train_scaled,x_used_train = predict_recursive_series(
    model=model,
    X_df=X_train_selected_df,
    output_cols=output_cols,
    X_feature_names=X_feature_names,
    na=na
        )

    print(f'-----predicting on Testing data----')
    
    y_pred_test_scaled , x_used_test=predict_recursive_series(
    model=model,
    X_df=X_test_selected_df,
    output_cols=output_cols,
    X_feature_names=X_feature_names,
    na=na
        )



    # Inverse transform to original scale
    y_pred_train = scaler_y_pitch.inverse_transform(y_pred_train_scaled)
    y_pred_test = scaler_y_pitch.inverse_transform(y_pred_test_scaled)


   

    # Convert predictions to DataFrames for easier handling
    y_pred_train_df = pd.DataFrame(y_pred_train, columns=output_cols)
    y_pred_test_df = pd.DataFrame(y_pred_test, columns=output_cols)


    # Get true values aligned with dfy_train and dfy_test indexes
    y_true_train =scaler_y_pitch.inverse_transform(dfy_train[output_cols].reset_index(drop=True))
    y_true_train_df = pd.DataFrame(y_true_train, columns=output_cols)

    y_true_test = scaler_y_pitch.inverse_transform(dfy_test[output_cols].reset_index(drop=True))
    y_true_test_df = pd.DataFrame(y_true_test, columns=output_cols)



    # save model info to dfs
    metrics_row = {
    'na': na,
    'nb': nb,
    'nf': nf,
    'xcase': xcase,
    'test case': case,
    'dt' : dt
    }

    
    print(f'-----Eavluating model----')
    # Compute metrics
    if np.any(np.isnan(y_pred_train_df)) or np.any(np.isinf(y_pred_train_df)):
        print("[WARN] Train predictions have NaNs or infs.")

    for col in output_cols:
        mse_train = mean_squared_error(y_true_train_df[col], y_pred_train_df[col])
        r2_train = r2_score(y_true_train_df[col], y_pred_train_df[col])

        mse_test = mean_squared_error(y_true_test_df[col], y_pred_test_df[col])
        r2_test = r2_score(y_true_test_df[col], y_pred_test_df[col])

        # Add to metrics row
        metrics_row[f'r2_train_{col}'] = r2_train
        metrics_row[f'mse_train_{col}'] = mse_train
        metrics_row[f'r2_test_{col}'] = r2_test
        metrics_row[f'mse_test_{col}'] = mse_test

    print("metrics_row:", metrics_row)
    # Append the row dictionaries as new rows in the DataFrames
    # Make sure metrics_row is a dict, not a list of dicts
    if isinstance(metrics_row, dict):
        row_df = pd.DataFrame([metrics_row])
    else:
        row_df = pd.DataFrame(metrics_row)

    metrics_df_ARX_picth_dt_senstivery_test = pd.concat([metrics_df_ARX_picth_dt_senstivery_test, row_df], ignore_index=True)

    



In [ ]:
# cheking unstable predictions
# Assuming y_true_train_df and y_pred_train_df are pandas DataFrames
for col in output_cols:
    plt.figure(figsize=(10, 6))  # Set the figure size
    plt.plot(y_true_val_df.index, y_true_val_df[col], label='True', linestyle='-', color='b')
    plt.plot(y_pred_val_df.index, y_pred_val_df[col], label='Predicted', linestyle='--', color='r')
    plt.title(f'True vs Predicted for {col}')
    plt.xlabel('Index')
    plt.ylabel(col)
    plt.legend()
    plt.grid(True)
    plt.show()


In [ ]:
metrics_df_ARX_picth_dt_senstivery_test

In [ ]:
# save results 
# Define your new folder path
save_folder = "ARX/metrics_outpus/dt_test"

# Base save folder
save_folder = 'Results/dt_tests'

# Create the directory if it doesn't exist
os.makedirs(save_folder, exist_ok=True)
# Define the file name
file_path = os.path.join(save_folder, "metrics_df_ARX_picth_dt_senstivery_test-.csv")

# Save the DataFrame
metrics_df_ARX_picth_dt_senstivery_test.to_csv(file_path, index=False)




parallel

In [ ]:
metrics_df_ARX_picth_dt_senstivery_test_parralel=pd.DataFrame()


In [ ]:
steps=[1,2,3,4,5,6]
case='Tp6p8s_Hs2m'

output_cols = ['pitch']
#for na in range(4,na_max+1):
na=2
nb=2
nf=7
degree=1
xcase = 'eta'  # xcases
input_cols=['eta' , 'eta_velocity', 'eta_acceleration']

In [ ]:

# prepare training data

dfx_train,dfy_train_full,yi_train = build_arx_lagged_with_scalers(
        df = df_case_train,
        input_cols  = input_cols,
        output_cols   = output_cols,
        scaler_X_func   = scaler_X_func_nov,
        scaler_y_func   = scaler_y_func_pitch,
        na=na,
        nb_past=nb,
        nf_future=nf,
        test_name_col='test_name',
        y_initial_mode='zero'  # 'original' ➔ skip initial rows, 'zero' ➔ pad lags with zeros
)
    
y_train_target_full = dfy_train_full[output_cols].reset_index(drop=True)



# prepare testing data

dfx_test,dfy_test_full,yi_test = build_arx_lagged_with_scalers(
        df = df_case_test,
        input_cols  = input_cols,
        output_cols   = output_cols,
        scaler_X_func   = scaler_X_func_nov,
        scaler_y_func   = scaler_y_func_pitch,
        na=na,
        nb_past=nb,
        nf_future=nf,
        test_name_col='test_name',
        y_initial_mode='original'  # 'original' ➔ skip initial rows, 'zero' ➔ pad lags with zeros
)


# select eta only in input features
X_train_selected_df_full = select_input_features(
    dfx_train.drop(columns='test_name'),
    input_type=xcase,
    keep_future=True,
    keep_past=True
)

X_test_selected_df_full = select_input_features(
    dfx_test.drop(columns='test_name'),
    input_type=xcase,
    keep_future=True,
    keep_past=True
)
    
for step in steps:
    
    dt=0.05*step
    
    
    # resample data
    
    X_train_selected_df= X_train_selected_df_full.iloc[::step].reset_index(drop=True)
    y_train_target= y_train_target_full.iloc[::step].reset_index(drop=True)
    X_test_selected_df= X_test_selected_df_full.iloc[::step].reset_index(drop=True)
    dfy_train= dfy_train_full.iloc[::step].reset_index(drop=True)
    dfy_test= dfy_test_full.iloc[::step].reset_index(drop=True)
    
    
    # Create the pipeline model
    model = Pipeline([
            ('polynomial', PolynomialFeatures(degree=degree, interaction_only=False)),
            ('linear', LinearRegression())
        ])
        
    # prepare training data 
    print(f'-----Preprocessing case of dt {dt} ----')
    # ============================
    # MEASURE CPU & MEMORY USAGE
    # ============================
    print(f'-----Training model----')
    
    model.fit(X_train_selected_df  ,y_train_target )
    


    # get feature names
    X_feature_names = X_train_selected_df.columns.tolist()


    print(f'-----predicting on training data----')

    # Predict on train and Testing data
    

    y_pred_train_scaled= model.predict(X_train_selected_df)

    print(f'-----predicting on Testing data----')
    
    y_pred_test_scaled=model.predict(X_test_selected_df)



    # Inverse transform to original scale
    y_pred_train = scaler_y_pitch.inverse_transform(y_pred_train_scaled)
    y_pred_test = scaler_y_pitch.inverse_transform(y_pred_test_scaled)


   

    # Convert predictions to DataFrames for easier handling
    y_pred_train_df = pd.DataFrame(y_pred_train, columns=output_cols)
    y_pred_test_df = pd.DataFrame(y_pred_test, columns=output_cols)


    # Get true values aligned with dfy_train and dfy_test indexes
    y_true_train =scaler_y_pitch.inverse_transform(dfy_train[output_cols].reset_index(drop=True))
    y_true_train_df = pd.DataFrame(y_true_train, columns=output_cols)

    y_true_test = scaler_y_pitch.inverse_transform(dfy_test[output_cols].reset_index(drop=True))
    y_true_test_df = pd.DataFrame(y_true_test, columns=output_cols)



    # save model info to dfs
    metrics_row = {
    'na': na,
    'nb': nb,
    'nf': nf,
    'xcase': xcase,
    'test case': case,
    'dt' : dt
    }

    
    print(f'-----Eavluating model----')
    # Compute metrics
    if np.any(np.isnan(y_pred_train_df)) or np.any(np.isinf(y_pred_train_df)):
        print("[WARN] Train predictions have NaNs or infs.")

    for col in output_cols:
        mse_train = mean_squared_error(y_true_train_df[col], y_pred_train_df[col])
        r2_train = r2_score(y_true_train_df[col], y_pred_train_df[col])

        mse_test = mean_squared_error(y_true_test_df[col], y_pred_test_df[col])
        r2_test = r2_score(y_true_test_df[col], y_pred_test_df[col])

        # Add to metrics row
        metrics_row[f'r2_train_{col}'] = r2_train
        metrics_row[f'mse_train_{col}'] = mse_train
        metrics_row[f'r2_test_{col}'] = r2_test
        metrics_row[f'mse_test_{col}'] = mse_test

    print("metrics_row:", metrics_row)
    # Append the row dictionaries as new rows in the DataFrames
    # Make sure metrics_row is a dict, not a list of dicts
    if isinstance(metrics_row, dict):
        row_df = pd.DataFrame([metrics_row])
    else:
        row_df = pd.DataFrame(metrics_row)

    metrics_df_ARX_picth_dt_senstivery_test_parralel = pd.concat([metrics_df_ARX_picth_dt_senstivery_test_parralel, row_df], ignore_index=True)

    



In [ ]:
metrics_df_ARX_picth_dt_senstivery_test_parralel

In [ ]:
# save results 
# Define your new folder path
save_folder = "ARX/metrics_outpus/dt_test"

# Base save folder
save_folder = 'Results/dt_tests'

# Create the directory if it doesn't exist
os.makedirs(save_folder, exist_ok=True)
# Define the file name
file_path = os.path.join(save_folder, "metrics_df_ARX_picth_dt_senstivery_test_parralel-.csv")

# Save the DataFrame
metrics_df_ARX_picth_dt_senstivery_test_parralel.to_csv(file_path, index=False)




## Data Senstivity

Parallel


In [ ]:
lenght=np.array([0.002,0.004,0.008,0.01,0.02 ,0.05 , 0.1 , 0.2 , 0.3 , 0.4 , 0.5 , 0.6 , 0.7 , 0.8 , 0.9,1])*len(df_case_train)
lenght=lenght.astype(int)

output_cols = ['pitch']
#for na in range(4,na_max+1):
na=2
nb=2
nf=7

xcase = 'eta'  # xcases
input_cols=['eta' , 'eta_velocity', 'eta_acceleration']

In [ ]:
# prepare training data

dfx_train,dfy_train,yi_train = build_arx_lagged_with_scalers(
        df = df_case_train,
        input_cols  = input_cols,
        output_cols   = output_cols,
        scaler_X_func   = scaler_X_func_nov,
        scaler_y_func   = scaler_y_func_pitch,
        na=na,
        nb_past=nb,
        nf_future=nf,
        test_name_col='test_name',
        y_initial_mode='zero'  # 'original' ➔ skip initial rows, 'zero' ➔ pad lags with zeros
)
    
y_train_target = dfy_train[output_cols].reset_index(drop=True)



# prepare Testing data

dfx_test,dfy_test,yi_test = build_arx_lagged_with_scalers(
        df = df_case_test,
        input_cols  = input_cols,
        output_cols   = output_cols,
        scaler_X_func   = scaler_X_func_nov,
        scaler_y_func   = scaler_y_func_pitch,
        na=na,
        nb_past=nb,
        nf_future=nf,
        test_name_col='test_name',
        y_initial_mode='original'  # 'original' ➔ skip initial rows, 'zero' ➔ pad lags with zeros
)


# select eta only in input features
X_train_selected_df = select_input_features(
    dfx_train.drop(columns='test_name'),
    input_type=xcase,
    keep_future=True,
    keep_past=True
)

X_test_selected_df = select_input_features(
    dfx_test.drop(columns='test_name'),
    input_type=xcase,
    keep_future=True,
    keep_past=True
)


In [ ]:
metrics_df_ARX_picth_data_test_parralel=pd.DataFrame()


In [ ]:

for l in lenght:
    
        # Create the pipeline model
    model = Pipeline([
            ('polynomial', PolynomialFeatures(degree=degree, interaction_only=False)),
            ('linear', LinearRegression())
        ])
        
    # prepare training data 
    print(f'-----Preprocessing case of lenght {str(l)} ----')
    # ============================
    # MEASURE CPU & MEMORY USAGE
    # ============================
    print(f'-----Training model----')
    
    model.fit(X_train_selected_df[0:l]  ,y_train_target[0:l] )
    


    # get feature names
    X_feature_names = X_train_selected_df.columns.tolist()


    print(f'-----predicting on training data----')

    # Predict on train and Testing data
    start_time = time.perf_counter()

    y_pred_train_scaled=model.predict(X_train_selected_df[0:l])

    print(f'-----predicting on Testing data----')
    
    y_pred_test_scaled = model.predict(X_test_selected_df)



    # Inverse transform to original scale
    y_pred_train = scaler_y_pitch.inverse_transform(y_pred_train_scaled)
    y_pred_test = scaler_y_pitch.inverse_transform(y_pred_test_scaled)


   

    # Convert predictions to DataFrames for easier handling
    y_pred_train_df = pd.DataFrame(y_pred_train, columns=output_cols)
    y_pred_test_df = pd.DataFrame(y_pred_test, columns=output_cols)


    # Get true values aligned with dfy_train and dfy_test indexes
    y_true_train =scaler_y_pitch.inverse_transform(dfy_train[output_cols][0:l].reset_index(drop=True))
    y_true_train_df = pd.DataFrame(y_true_train, columns=output_cols)

    y_true_test = scaler_y_pitch.inverse_transform(dfy_test[output_cols].reset_index(drop=True))
    y_true_test_df = pd.DataFrame(y_true_test, columns=output_cols)



    # save model info to dfs
    metrics_row = {
    'model_name': model_name,
    'na': na,
    'nb': nb,
    'nf': nf,
    'xcase': xcase,
    'test case': case,
    'lenght' : l
    }

    
    print(f'-----Eavluating model----')
    # Compute metrics
    if np.any(np.isnan(y_pred_train_df)) or np.any(np.isinf(y_pred_train_df)):
        print("[WARN] Train predictions have NaNs or infs.")

    for col in output_cols:
        mse_train = mean_squared_error(y_true_train_df[col], y_pred_train_df[col])
        r2_train = r2_score(y_true_train_df[col], y_pred_train_df[col])

        mse_test = mean_squared_error(y_true_test_df[col], y_pred_test_df[col])
        r2_test = r2_score(y_true_test_df[col], y_pred_test_df[col])

        # Add to metrics row
        metrics_row[f'r2_train_{col}'] = r2_train
        metrics_row[f'mse_train_{col}'] = mse_train
        metrics_row[f'r2_test_{col}'] = r2_test
        metrics_row[f'mse_test_{col}'] = mse_test

    print("metrics_row:", metrics_row)
    # Append the row dictionaries as new rows in the DataFrames
    # Make sure metrics_row is a dict, not a list of dicts
    if isinstance(metrics_row, dict):
        row_df = pd.DataFrame([metrics_row])
    else:
        row_df = pd.DataFrame(metrics_row)

    metrics_df_ARX_picth_data_test_parralel = pd.concat([metrics_df_ARX_picth_data_test_parralel, row_df], ignore_index=True)




In [ ]:
# Base save folder
save_folder = 'Results/data_tests'

# Create the directory if it doesn't exist
os.makedirs(save_folder, exist_ok=True)
# Define the file name
file_path = os.path.join(save_folder, "metrics_df_ARX_picth_data_test_parralel-.csv")

# Save the DataFrame
metrics_df_ARX_picth_data_test_parralel.to_csv(file_path, index=False)



series

In [ ]:
metrics_df_ARX_picth_data_test_seiries=pd.DataFrame()

In [ ]:
for l in lenght:
    
        # Create the pipeline model
    model = Pipeline([
            ('polynomial', PolynomialFeatures(degree=degree, interaction_only=False)),
            ('linear', LinearRegression())
        ])
        
    # prepare training data 
    print(f'-----Preprocessing case of lenght {str(l)} ----')
    # ============================
    # MEASURE CPU & MEMORY USAGE
    # ============================
    print(f'-----Training model----')
    
    model.fit(X_train_selected_df[0:l]  ,y_train_target[0:l] )
    


    # get feature names
    X_feature_names = X_train_selected_df.columns.tolist()


    print(f'-----predicting on training data----')

    # Predict on train and Testing data
    start_time = time.perf_counter()

    y_pred_train_scaled,x_used_train = predict_recursive_series(
    model=model,
    X_df=X_train_selected_df[0:l],
    output_cols=output_cols,
    X_feature_names=X_feature_names,
    na=na
        )

    print(f'-----predicting on Testing data----')
    
    y_pred_test_scaled , x_used_test=predict_recursive_series(
    model=model,
    X_df=X_test_selected_df,
    output_cols=output_cols,
    X_feature_names=X_feature_names,
    na=na
        )



    # Inverse transform to original scale
    y_pred_train = scaler_y_pitch.inverse_transform(y_pred_train_scaled)
    y_pred_test = scaler_y_pitch.inverse_transform(y_pred_test_scaled)


   

    # Convert predictions to DataFrames for easier handling
    y_pred_train_df = pd.DataFrame(y_pred_train, columns=output_cols)
    y_pred_test_df = pd.DataFrame(y_pred_test, columns=output_cols)


    # Get true values aligned with dfy_train and dfy_test indexes
    y_true_train =scaler_y_pitch.inverse_transform(dfy_train[output_cols][0:l].reset_index(drop=True))
    y_true_train_df = pd.DataFrame(y_true_train, columns=output_cols)

    y_true_test = scaler_y_pitch.inverse_transform(dfy_test[output_cols].reset_index(drop=True))
    y_true_test_df = pd.DataFrame(y_true_test, columns=output_cols)



    # save model info to dfs
    metrics_row = {
    'model_name': model_name,
    'na': na,
    'nb': nb,
    'nf': nf,
    'xcase': xcase,
    'test case': case,
    'lenght' : l
    }

    
    print(f'-----Eavluating model----')
    # Compute metrics
    if np.any(np.isnan(y_pred_train_df)) or np.any(np.isinf(y_pred_train_df)):
        print("[WARN] Train predictions have NaNs or infs.")

    for col in output_cols:
        mse_train = mean_squared_error(y_true_train_df[col], y_pred_train_df[col])
        r2_train = r2_score(y_true_train_df[col], y_pred_train_df[col])

        mse_test = mean_squared_error(y_true_test_df[col], y_pred_test_df[col])
        r2_test = r2_score(y_true_test_df[col], y_pred_test_df[col])

        # Add to metrics row
        metrics_row[f'r2_train_{col}'] = r2_train
        metrics_row[f'mse_train_{col}'] = mse_train
        metrics_row[f'r2_test_{col}'] = r2_test
        metrics_row[f'mse_test_{col}'] = mse_test

    print("metrics_row:", metrics_row)
    # Append the row dictionaries as new rows in the DataFrames
    # Make sure metrics_row is a dict, not a list of dicts
    if isinstance(metrics_row, dict):
        row_df = pd.DataFrame([metrics_row])
    else:
        row_df = pd.DataFrame(metrics_row)

    metrics_df_ARX_picth_data_test_seiries = pd.concat([metrics_df_ARX_picth_data_test_seiries, row_df], ignore_index=True)




In [ ]:
# Base save folder
save_folder = 'Results/data_tests'

# Create the directory if it doesn't exist
os.makedirs(save_folder, exist_ok=True)
# Define the file name
file_path = os.path.join(save_folder, "metrics_df_ARX_picth_data_test_seiries-.csv")

# Save the DataFrame
metrics_df_ARX_picth_data_test_seiries.to_csv(file_path, index=False)



## Testing

In [ ]:
# now get the best model and predict on the test data
# Define the best model parameters
best_na = 2
best_nb = 2
best_nf = 7
best_degree = 1
best_xcase = 'eta'
best_input_cols = ['eta', 'eta_velocity', 'eta_acceleration']
best_output_cols = ['pitch']
# load the best model
folder='ARX/choosen heros/pitch'
model_name = f"ARX_pitcheta_na2_nb2_nf7.joblib"
# Load the model
model_path = os.path.join(folder, model_name)  
model = joblib.load(model_path)

# print all test cases
test_cases = df_test_full['test_name'].unique()
print("All test cases:")
print(test_cases)  



In [ ]:
input_cols = ['eta']  # original inputs
output_cols = ['pitch']  # outputs

# Construct feature names in the exact order used by the data pipeline
feature_names = []

# 1. Current input signals (no lag)
feature_names += input_cols

# 2. Output lags (autoregressive part)
for lag in range(1, best_na + 1):
    for out_col in output_cols:
        feature_names.append(f"{out_col}_lag_{lag}")

# 3. Past input lags
for lag in range(1, best_nb + 1):
    for in_col in input_cols:
        feature_names.append(f"{in_col}_past_{lag}")

# 4. Future input lags
for lag in range(1, best_nf + 1):
    for in_col in input_cols:
        feature_names.append(f"{in_col}_future_{lag}")

# View the final ordered feature list
print("Feature names used in X_lagged_df:")
print(feature_names)

In [ ]:
poly = model.named_steps['polynomial']
linear = model.named_steps['linear']

# Use the names remembered from training
input_names = poly.get_feature_names_out(poly.feature_names_in_)

# Coefficients and intercept
coefficients = linear.coef_.flatten()
intercept = float(linear.intercept_)


# Build equation
equation_terms = [f"{coef:.4f} * {name}" for coef, name in zip(coefficients, input_names)]
equation = f"{intercept:.4f} + " + " + ".join(equation_terms)

print("\n📘 Model Equation:")
print(f"y = {equation}")


In [ ]:
metrics_df_test_pitch_nb2_nf7 = pd.DataFrame()  # Will collect all test metrics
training_case='Tp6p8s_Hs2m'

In [ ]:
cases=df_test_full['test_name'].unique()
for case in cases:
    # Filter the test data
    df_case_test = df_test_full[df_test_full['test_name'] == case].reset_index(drop=True)
    # Prepare the test data
    dfx_test, dfy_test, yi_test = build_arx_lagged_with_scalers(
        df=df_case_test,
        input_cols=best_input_cols,
        output_cols=best_output_cols,
        scaler_X_func   = scaler_X_func_nov,
        scaler_y_func   = scaler_y_func_pitch,
        na=best_na,
        nb_past=best_nb,
        nf_future=best_nf,
        test_name_col='test_name',
        y_initial_mode='original'  # 'original' ➔ skip initial rows, 'zero' ➔ pad lags with zeros
    )

    # Select only the best input features
    X_test_selected_df = select_input_features(
        dfx_test.drop(columns='test_name'),
        input_type=best_xcase,
        keep_future=True,
        keep_past=True
    )

    # get feature names from model
    X_feature_names = X_test_selected_df.columns.tolist()

    # Predict on test data
    y_pred_test_scaled, x_used_test = predict_recursive_series(
        model=model,
        X_df=X_test_selected_df,
        output_cols=best_output_cols,
        X_feature_names=X_feature_names,
        na=best_na
    )

    # Inverse transform to original scale
    y_pred_test = scaler_y_pitch.inverse_transform(y_pred_test_scaled)
    # Convert to DataFrame
    y_pred_test_df = pd.DataFrame(y_pred_test, columns=best_output_cols)

    # Get true values aligned with dfy_test indexes
    y_true_test = scaler_y_pitch.inverse_transform(dfy_test[best_output_cols].reset_index(drop=True))
    y_true_test_df = pd.DataFrame(y_true_test, columns=best_output_cols)

    # Compute metrics
    metrics_test = {}
    metrics_test[f'test case'] = case
    if case == training_case:
        metrics_test[f'Comments'] = 'case used for training'
   
    for col in best_output_cols:
        mse_test = mean_squared_error(y_true_test_df[col], y_pred_test_df[col])
        r2_test = r2_score(y_true_test_df[col], y_pred_test_df[col])

        
        metrics_test[f'r2_test_{col}'] = r2_test
        metrics_test[f'mse_test_{col}'] = mse_test
    
    print("metrics:", metrics_test)
    # Append the row dictionaries as new rows in the DataFrame
    metrics_df_test_pitch_nb2_nf7  = pd.concat([metrics_df_test_pitch_nb2_nf7, pd.DataFrame([metrics_test])], ignore_index=True)
        # Save true and predicted DataFrames to CSV files
    y_true_test_df.to_csv(f'Results/ARX/Testing/heave/y_true_test_pitch_{case}.csv', index=False)
    y_pred_test_df.to_csv(f'Results/ARX/Testing/heave/y_pred_test_pitch_{case}.csv', index=False)



In [ ]:
# save results 
# Define your new folder path
save_folder = "ARX/best_model/metrics_outpus"

# Create the folder if it doesn't exist
os.makedirs(save_folder, exist_ok=True)
# Define the file name
file_path = os.path.join(save_folder, "metrics_df_test_pitch_nb2_nf7.csv")

# Save the DataFrame
metrics_df_test_pitch_nb2_nf7.to_csv(file_path, index=False)

print(f"Saved metrics_df_test_pitch_nb2_nf7 to: {file_path}")

## Noisy data test

In [ ]:
# now get the best model and predict on the test data
# Define the best model parameters
best_na = 2
best_nb = 2
best_nf = 7
best_degree = 1
best_xcase = 'eta'
best_input_cols = ['eta', 'eta_velocity', 'eta_acceleration']
best_output_cols = ['pitch']
# load the best model
folder='ARX/choosen heros/pitch'
model_name = f"ARX_pitcheta_na2_nb2_nf7_new.joblib"
# Load the model
model_path = os.path.join(folder, model_name)  
model = joblib.load(model_path)


In [ ]:
# load Noisy data
df_case_test_noisy=pd.read_csv('Results/df_case_test_noisy.csv')
df_case_test_noisy

In [ ]:
# peparing test data

dfx_test, dfy_test, yi_test = build_arx_lagged_with_scalers(
    df=df_case_test_noisy,
    input_cols=best_input_cols,
    output_cols=best_output_cols,
    scaler_X_func   = scaler_X_func_nov,
    scaler_y_func   = scaler_y_func_pitch,
    na=best_na,
    nb_past=best_nb,
    nf_future=best_nf,
    test_name_col='test_name',
    y_initial_mode='original'  # 'original' ➔ skip initial rows, 'zero' ➔ pad lags with zeros
)

# Select only the best input features
X_test_selected_df = select_input_features(
    dfx_test.drop(columns='test_name'),
    input_type=best_xcase,
    keep_future=True,
    keep_past=True
)

# get feature names from model
X_feature_names = X_test_selected_df.columns.tolist()

# Predict on test data
y_pred_test_scaled, x_used_test = predict_recursive_series(
    model=model,
    X_df=X_test_selected_df,
    output_cols=best_output_cols,
    X_feature_names=X_feature_names,
    na=best_na
)

# Inverse transform to original scale
y_pred_test = scaler_y_pitch.inverse_transform(y_pred_test_scaled)
# Convert to DataFrame
y_pred_test_df = pd.DataFrame(y_pred_test, columns=best_output_cols)

# Get true values aligned with dfy_test indexes
y_true_test = scaler_y_pitch.inverse_transform(dfy_test[best_output_cols].reset_index(drop=True))
y_true_test_df = pd.DataFrame(y_true_test, columns=best_output_cols)

# Compute metrics
metrics_test = {}
metrics_test[f'test case'] = case


mse_test = mean_squared_error(y_true_test_df['pitch'], y_pred_test_df['pitch'])
r2_test = r2_score(y_true_test_df['pitch'], y_pred_test_df['pitch'])


metrics_test[f'r2_test_pitch'] = r2_test
metrics_test[f'mse_test_pitch'] = mse_test

print("metrics:", metrics_test)

In [ ]:
y_true_test_df.to_csv(f'Results/noisy test/ARX_pitch_true.csv', index=False)
y_pred_test_df.to_csv(f'Results/noisy test/ARX_pitch_pred.csv', index=False)

# 1 model for Pendulum only

## Finding Oprimal Lags

In [ ]:
# Initialize empty DataFrames before the loop
metrics_df_ARX_pend_only_new_na2 = pd.DataFrame()
perf_df_ARX_series_pend_only_new_na2 = pd.DataFrame()

# Base save folder
save_folder = 'ARX/saved_models/new'

# Create the directory if it doesn't exist
os.makedirs(save_folder, exist_ok=True)


In [ ]:
# define loop values 
case='Tp6p8s_Hs2m'
na_max  = [2]
nb_max= [4]
nf_max=[2]
degree = 1
xcase = 'eta'  # xcases
input_cols=['eta' , 'eta_velocity', 'eta_acceleration']
output_cols = ['pendulum']

In [ ]:
for na in na_max:
    for nb in nb_max:
        for nf in nf_max:
            
            # Model name
            model_name = F'ARX_pend' + xcase + '_na' + str(na) + '_nb' + str(nb) + '_nf' + str(nf) + 'new'


            # Create the pipeline model
            model = Pipeline([
                    ('polynomial', PolynomialFeatures(degree=degree, interaction_only=False)),
                    ('linear', LinearRegression())
                ])
                
            # prepare training data 
            print(f'-----Preprocessing case of na={str(na)} ,nb={str(nb)} and nf={str(nf)} , with xcase: {xcase}----')
            # get  df at true scale witAh only this test case
            df_case_train=df_train_full[df_train_full['test_name']==case].reset_index(drop=True)



            dfx_train,dfy_train,yi_train = build_arx_lagged_with_scalers(
                    df = df_case_train,
                    input_cols  = input_cols,
                    output_cols   = output_cols,
                    scaler_X_func   = scaler_X_func_nov,
                    scaler_y_func   = scaler_y_func_pend,
                    na=na,
                    nb_past=nb,
                    nf_future=nf,
                    test_name_col='test_name',
                    y_initial_mode='zero'  # 'original' ➔ skip initial rows, 'zero' ➔ pad lags with zeros
            )
                
            y_train_target = dfy_train[output_cols].reset_index(drop=True)



            # prepare validation data
            df_case_val=df_val_full[df_val_full['test_name']==case].reset_index(drop=True)

            dfx_val,dfy_val,yi_val = build_arx_lagged_with_scalers(
                    df = df_case_val,
                    input_cols  = input_cols,
                    output_cols   = output_cols,
                    scaler_X_func   = scaler_X_func_nov,
                    scaler_y_func   = scaler_y_func_pend,
                    na=na,
                    nb_past=nb,
                    nf_future=nf,
                    test_name_col='test_name',
                    y_initial_mode='original'  # 'original' ➔ skip initial rows, 'zero' ➔ pad lags with zeros
            )


            # select eta only in input features
            X_train_selected_df = select_input_features(
                dfx_train.drop(columns='test_name'),
                input_type=xcase,
                keep_future=True,
                keep_past=True
            )

            X_val_selected_df = select_input_features(
                dfx_val.drop(columns='test_name'),
                input_type=xcase,
                keep_future=True,
                keep_past=True
            )

            # ============================
            # MEASURE CPU & MEMORY USAGE
            # ============================
            print(f'-----Training model----')
            process = psutil.Process()

            # Train the model
            start_time = time.perf_counter()
            model.fit(X_train_selected_df  ,y_train_target  )
            train_time = time.perf_counter() - start_time
            memory_usage_train = process.memory_info().rss / (1024 * 1024)  # MB


            # get feature names
            X_feature_names = X_train_selected_df.columns.tolist()


            print(f'-----predicting on training data----')

            # Predict on train and validation data
            start_time = time.perf_counter()

            y_pred_train_scaled,x_used_train = predict_recursive_series(
            model=model,
            X_df=X_train_selected_df,
            output_cols=output_cols,
            X_feature_names=X_feature_names,
            na=na
                )

            print(f'-----predicting on validation data----')
            
            y_pred_val_scaled , x_used_val=predict_recursive_series(
            model=model,
            X_df=X_val_selected_df,
            output_cols=output_cols,
            X_feature_names=X_feature_names,
            na=na
                )



            # Inverse transform to original scale
            y_pred_train = scaler_y_pend.inverse_transform(y_pred_train_scaled)
            y_pred_val = scaler_y_pend.inverse_transform(y_pred_val_scaled)


            predict_time = time.perf_counter() - start_time
            memory_usage_predict = process.memory_info().rss / (1024 * 1024)  # MB

            # Convert predictions to DataFrames for easier handling
            y_pred_train_df = pd.DataFrame(y_pred_train, columns=output_cols)
            y_pred_val_df = pd.DataFrame(y_pred_val, columns=output_cols)


            # Get true values aligned with dfy_train and dfy_val indexes
            y_true_train =scaler_y_pend.inverse_transform(dfy_train[output_cols].reset_index(drop=True))
            y_true_train_df = pd.DataFrame(y_true_train, columns=output_cols)

            y_true_val = scaler_y_pend.inverse_transform(dfy_val[output_cols].reset_index(drop=True))
            y_true_val_df = pd.DataFrame(y_true_val, columns=output_cols)



            # save model info to dfs
            metrics_row = {
            'model_name': model_name,
            'na': na,
            'nb': nb,
            'nf': nf,
            'xcase': xcase,
            'test case': case,
            }

            perf_row = {
            'model_name': model_name,
            'na': na,
            'nb': nb,
            'nf': nf,
            'degree': degree,
            'xcase': xcase,
            'train_time': train_time,
            'train_memory_MB': memory_usage_train,
            'predict_time': predict_time,
            'predict_memory_MB': memory_usage_predict
                }
            print(f'-----Eavluating model----')
            # Compute metrics
            if np.any(np.isnan(y_pred_train_df)) or np.any(np.isinf(y_pred_train_df)):
                print("[WARN] Train predictions have NaNs or infs.")

            for col in output_cols:
                mse_train = mean_squared_error(y_true_train_df[col], y_pred_train_df[col])
                r2_train = r2_score(y_true_train_df[col], y_pred_train_df[col])

                mse_val = mean_squared_error(y_true_val_df[col], y_pred_val_df[col])
                r2_val = r2_score(y_true_val_df[col], y_pred_val_df[col])

                # Add to metrics row
                metrics_row[f'r2_train_{col}'] = r2_train
                metrics_row[f'mse_train_{col}'] = mse_train
                metrics_row[f'r2_val_{col}'] = r2_val
                metrics_row[f'mse_val_{col}'] = mse_val

            print("metrics_row:", metrics_row)
            print("perf_row:", perf_row)
            # Append the row dictionaries as new rows in the DataFrames
            metrics_df_ARX_pend_only_final  = pd.concat([metrics_df_ARX_pend_only_final, pd.DataFrame([metrics_row])], ignore_index=True)
            perf_df_ARX_series_pend_only_new_na2 = pd.concat([perf_df_ARX_series_pend_only_new_na2, pd.DataFrame([perf_row])], ignore_index=True)
            print(f'-----Saving model----')
            # Save the trained model
            model_save_name = f"{model_name}.joblib"  # You already have model_name variable!
            model_save_path = os.path.join(save_folder, model_save_name)

            # Save with joblib
            joblib.dump(model, model_save_path) 




In [ ]:
# save results 
# Define your new folder path
save_folder = "ARX/metrics_outpus/pendulum"

# Create the folder if it doesn't exist
os.makedirs(save_folder, exist_ok=True)
# Define the file name
file_path = os.path.join(save_folder, "metrics_df_ARX_pend_only_final.csv")

# Save the DataFrame
perf_df_ARX_series_pend_only_new_na2.to_csv(file_path, index=False)
print(f"Saved metrics_df_ARX_pend_only_final to: {file_path}")

In [ ]:
# load the saved metrics
metrics_df_ARX_pend_only= pd.read_csv(r"D:\thesis\Work-space\ARX\metrics_outpus\pendulum\metrics_df_ARX_pend_only_final.csv")

In [ ]:
# Set target output variable
target_var = 'pendulum'
df_plot = metrics_df_ARX_pend_only.copy()

# Unique values
na_values = sorted(df_plot['na'].unique())
nf_values = sorted(df_plot['nf'].unique())

# Color map to keep consistent colors for each nf
color_map = {
    nf: px.colors.qualitative.Plotly[i % len(px.colors.qualitative.Plotly)]
    for i, nf in enumerate(nf_values)
}

for na_val in na_values:
    df_na = df_plot[df_plot['na'] == na_val].copy()

    # Create subplot figure
    fig = make_subplots(
        rows=1, cols=2,
        subplot_titles=[f"R² Train (na={na_val})", f"R² Val (na={na_val})"],
        shared_yaxes=False
    )

    for nf_val in nf_values:
        df_nf = df_na[df_na['nf'] == nf_val].copy().sort_values(by='nb')
        color = color_map[nf_val]

        # Highlight nf=7
        if nf_val == 2:
            line_width = 4
            marker_size = 12
            marker_symbol = 'star'
        else:
            line_width = 2
            marker_size = 8
            marker_symbol = 'circle'

        # --- Train R² ---
        fig.add_trace(
            go.Scatter(
                x=df_nf['nb'],
                y=df_nf[f'r2_train_{target_var}'],
                mode='lines+markers',
                name=f'nd={-nf_val}',
                legendgroup=f'nd={-nf_val}',
                line=dict(color=color, width=line_width),
                marker=dict(size=marker_size, symbol=marker_symbol)
            ),
            row=1, col=1
        )

        # --- Val R² ---
        fig.add_trace(
            go.Scatter(
                x=df_nf['nb'],
                y=df_nf[f'r2_val_{target_var}'],
                mode='lines+markers',
                name=f'nd={-nf_val}',
                legendgroup=f'nd={-nf_val}',
                showlegend=False,
                line=dict(color=color, width=line_width),
                marker=dict(size=marker_size, symbol=marker_symbol)
            ),
            row=1, col=2
        )

    # Layout settings
    fig.update_layout(
        title=dict(
            text=f"R² Performance for na = {na_val} ({target_var})"
        ),
        xaxis_title="nb ",
        xaxis2_title="nb ",
        yaxis_title="R² Score",
        template="plotly_white",
        height=500,
        width=1125,
        legend_title="nd",
        yaxis1=dict(range=[0.5, 1]),
        yaxis2=dict(range=[0.5, 1]),
    )

    fig.show()


Best Model is at na 2 , nb 4 ,nf 2

## dt senstivety 

Series

In [ ]:
metrics_df_ARX_pendulum_dt_senstivery_test=pd.DataFrame()


In [ ]:
steps=[1,2,3,4,5,6]
case='Tp6p8s_Hs2m'

output_cols = ['pendulum']
#for na in range(4,na_max+1):
na=2
nb=4
nf=2
degree=1
xcase = 'eta'  # xcases
input_cols=['eta' , 'eta_velocity', 'eta_acceleration']

In [ ]:

##
# Model parameters

# get  df at true scale witAh only this test case
df_case_train=df_train_full[df_train_full['test_name']==case].reset_index(drop=True)



dfx_train,dfy_train_full,yi_train = build_arx_lagged_with_scalers(
        df = df_case_train,
        input_cols  = input_cols,
        output_cols   = output_cols,
        scaler_X_func   = scaler_X_func_nov,
        scaler_y_func   = scaler_y_func_pend,
        na=na,
        nb_past=nb,
        nf_future=nf,
        test_name_col='test_name',
        y_initial_mode='zero'  # 'original' ➔ skip initial rows, 'zero' ➔ pad lags with zeros
)
    
y_train_target_full = dfy_train_full[output_cols].reset_index(drop=True)



# prepare Testing data

dfx_test,dfy_test_full,yi_test = build_arx_lagged_with_scalers(
        df = df_case_test,
        input_cols  = input_cols,
        output_cols   = output_cols,
        scaler_X_func   = scaler_X_func_nov,
        scaler_y_func   = scaler_y_func_pend,
        na=na,
        nb_past=nb,
        nf_future=nf,
        test_name_col='test_name',
        y_initial_mode='original'  # 'original' ➔ skip initial rows, 'zero' ➔ pad lags with zeros
)


# select eta only in input features
X_train_selected_df_full = select_input_features(
    dfx_train.drop(columns='test_name'),
    input_type=xcase,
    keep_future=True,
    keep_past=True
)

X_test_selected_df_full = select_input_features(
    dfx_test.drop(columns='test_name'),
    input_type=xcase,
    keep_future=True,
    keep_past=True
)
    
for step in steps:
    
    dt=0.05*step
    
    
    # resample data
    
    X_train_selected_df= X_train_selected_df_full.iloc[::step].reset_index(drop=True)
    y_train_target= y_train_target_full.iloc[::step].reset_index(drop=True)
    X_test_selected_df= X_test_selected_df_full.iloc[::step].reset_index(drop=True)
    dfy_train= dfy_train_full.iloc[::step].reset_index(drop=True)
    dfy_test= dfy_test_full.iloc[::step].reset_index(drop=True)
    
    
    # Create the pipeline model
    model = Pipeline([
            ('polynomial', PolynomialFeatures(degree=degree, interaction_only=False)),
            ('linear', LinearRegression())
        ])
        
    # prepare training data 
    print(f'-----Preprocessing case of dt {dt} ----')
    # ============================
    # MEASURE CPU & MEMORY USAGE
    # ============================
    print(f'-----Training model----')
    
    model.fit(X_train_selected_df  ,y_train_target )
    


    # get feature names
    X_feature_names = X_train_selected_df.columns.tolist()


    print(f'-----predicting on training data----')

    # Predict on train and Testing data
    start_time = time.perf_counter()

    y_pred_train_scaled,x_used_train = predict_recursive_series(
    model=model,
    X_df=X_train_selected_df,
    output_cols=output_cols,
    X_feature_names=X_feature_names,
    na=na
        )

    print(f'-----predicting on Testing data----')
    
    y_pred_test_scaled , x_used_test=predict_recursive_series(
    model=model,
    X_df=X_test_selected_df,
    output_cols=output_cols,
    X_feature_names=X_feature_names,
    na=na
        )



    # Inverse transform to original scale
    y_pred_train = scaler_y_pend.inverse_transform(y_pred_train_scaled)
    y_pred_test = scaler_y_pend.inverse_transform(y_pred_test_scaled)


   

    # Convert predictions to DataFrames for easier handling
    y_pred_train_df = pd.DataFrame(y_pred_train, columns=output_cols)
    y_pred_test_df = pd.DataFrame(y_pred_test, columns=output_cols)


    # Get true values aligned with dfy_train and dfy_test indexes
    y_true_train =scaler_y_pend.inverse_transform(dfy_train[output_cols].reset_index(drop=True))
    y_true_train_df = pd.DataFrame(y_true_train, columns=output_cols)

    y_true_test = scaler_y_pend.inverse_transform(dfy_test[output_cols].reset_index(drop=True))
    y_true_test_df = pd.DataFrame(y_true_test, columns=output_cols)



    # save model info to dfs
    metrics_row = {
    'na': na,
    'nb': nb,
    'nf': nf,
    'xcase': xcase,
    'test case': case,
    'dt' : dt
    }

    
    print(f'-----Eavluating model----')
    # Compute metrics
    if np.any(np.isnan(y_pred_train_df)) or np.any(np.isinf(y_pred_train_df)):
        print("[WARN] Train predictions have NaNs or infs.")

    for col in output_cols:
        mse_train = mean_squared_error(y_true_train_df[col], y_pred_train_df[col])
        r2_train = r2_score(y_true_train_df[col], y_pred_train_df[col])

        mse_test = mean_squared_error(y_true_test_df[col], y_pred_test_df[col])
        r2_test = r2_score(y_true_test_df[col], y_pred_test_df[col])

        # Add to metrics row
        metrics_row[f'r2_train_{col}'] = r2_train
        metrics_row[f'mse_train_{col}'] = mse_train
        metrics_row[f'r2_test_{col}'] = r2_test
        metrics_row[f'mse_test_{col}'] = mse_test

    print("metrics_row:", metrics_row)
    # Append the row dictionaries as new rows in the DataFrames
    # Make sure metrics_row is a dict, not a list of dicts
    if isinstance(metrics_row, dict):
        row_df = pd.DataFrame([metrics_row])
    else:
        row_df = pd.DataFrame(metrics_row)

    metrics_df_ARX_pendulum_dt_senstivery_test = pd.concat([metrics_df_ARX_pendulum_dt_senstivery_test, row_df], ignore_index=True)


In [ ]:
# Base save folder
save_folder = 'Results/dt_tests'

# Create the directory if it doesn't exist
os.makedirs(save_folder, exist_ok=True)
# Define the file name
file_path = os.path.join(save_folder, "metrics_df_ARX_pendulum_dt_senstivery_test-.csv")

# Save the DataFrame
metrics_df_ARX_pendulum_dt_senstivery_test.to_csv(file_path, index=False)



Parallel

In [ ]:
metrics_df_ARX_pendulum_dt_senstivery_test_parralel=pd.DataFrame()


In [ ]:
steps=[1,2,3,4,5,6]
case='Tp6p8s_Hs2m'

output_cols = ['pendulum']
#for na in range(4,na_max+1):
na=2
nb=4
nf=2
degree=1
xcase = 'eta'  # xcases
input_cols=['eta' , 'eta_velocity', 'eta_acceleration']

In [ ]:
# preparing training data

dfx_train,dfy_train_full,yi_train = build_arx_lagged_with_scalers(
        df = df_case_train,
        input_cols  = input_cols,
        output_cols   = output_cols,
        scaler_X_func   = scaler_X_func_nov,
        scaler_y_func   = scaler_y_func_pend,
        na=na,
        nb_past=nb,
        nf_future=nf,
        test_name_col='test_name',
        y_initial_mode='zero'  # 'original' ➔ skip initial rows, 'zero' ➔ pad lags with zeros
)
    
y_train_target_full = dfy_train_full[output_cols].reset_index(drop=True)



# prepare Testing data

dfx_test,dfy_test_full,yi_test = build_arx_lagged_with_scalers(
        df = df_case_test,
        input_cols  = input_cols,
        output_cols   = output_cols,
        scaler_X_func   = scaler_X_func_nov,
        scaler_y_func   = scaler_y_func_pend,
        na=na,
        nb_past=nb,
        nf_future=nf,
        test_name_col='test_name',
        y_initial_mode='original'  # 'original' ➔ skip initial rows, 'zero' ➔ pad lags with zeros
)


# select eta only in input features
X_train_selected_df_full = select_input_features(
    dfx_train.drop(columns='test_name'),
    input_type=xcase,
    keep_future=True,
    keep_past=True
)

X_test_selected_df_full = select_input_features(
    dfx_test.drop(columns='test_name'),
    input_type=xcase,
    keep_future=True,
    keep_past=True
)
    
for step in steps:
    
    dt=0.05*step
    
    
    # resample data
    
    X_train_selected_df= X_train_selected_df_full.iloc[::step].reset_index(drop=True)
    y_train_target= y_train_target_full.iloc[::step].reset_index(drop=True)
    X_test_selected_df= X_test_selected_df_full.iloc[::step].reset_index(drop=True)
    dfy_train= dfy_train_full.iloc[::step].reset_index(drop=True)
    dfy_test= dfy_test_full.iloc[::step].reset_index(drop=True)
    
    
    # Create the pipeline model
    model = Pipeline([
            ('polynomial', PolynomialFeatures(degree=degree, interaction_only=False)),
            ('linear', LinearRegression())
        ])
        
    # prepare training data 
    print(f'-----Preprocessing case of dt {dt} ----')
    # ============================
    # MEASURE CPU & MEMORY USAGE
    # ============================
    print(f'-----Training model----')
    
    model.fit(X_train_selected_df  ,y_train_target )
    


    # get feature names
    X_feature_names = X_train_selected_df.columns.tolist()


    print(f'-----predicting on training data----')

    # Predict on train and Testing data
    start_time = time.perf_counter()

    y_pred_train_scaled=model.predict(X_train_selected_df)

    print(f'-----predicting on Testing data----')
    
    y_pred_test_scaled =model.predict(X_test_selected_df)        



    # Inverse transform to original scale
    y_pred_train = scaler_y_pend.inverse_transform(y_pred_train_scaled)
    y_pred_test = scaler_y_pend.inverse_transform(y_pred_test_scaled)


   

    # Convert predictions to DataFrames for easier handling
    y_pred_train_df = pd.DataFrame(y_pred_train, columns=output_cols)
    y_pred_test_df = pd.DataFrame(y_pred_test, columns=output_cols)


    # Get true values aligned with dfy_train and dfy_test indexes
    y_true_train =scaler_y_pend.inverse_transform(dfy_train[output_cols].reset_index(drop=True))
    y_true_train_df = pd.DataFrame(y_true_train, columns=output_cols)

    y_true_test = scaler_y_pend.inverse_transform(dfy_test[output_cols].reset_index(drop=True))
    y_true_test_df = pd.DataFrame(y_true_test, columns=output_cols)



    # save model info to dfs
    metrics_row = {
    'na': na,
    'nb': nb,
    'nf': nf,
    'xcase': xcase,
    'test case': case,
    'dt' : dt
    }

    
    print(f'-----Eavluating model----')
    # Compute metrics
    if np.any(np.isnan(y_pred_train_df)) or np.any(np.isinf(y_pred_train_df)):
        print("[WARN] Train predictions have NaNs or infs.")

    for col in output_cols:
        mse_train = mean_squared_error(y_true_train_df[col], y_pred_train_df[col])
        r2_train = r2_score(y_true_train_df[col], y_pred_train_df[col])

        mse_test = mean_squared_error(y_true_test_df[col], y_pred_test_df[col])
        r2_test = r2_score(y_true_test_df[col], y_pred_test_df[col])

        # Add to metrics row
        metrics_row[f'r2_train_{col}'] = r2_train
        metrics_row[f'mse_train_{col}'] = mse_train
        metrics_row[f'r2_test_{col}'] = r2_test
        metrics_row[f'mse_test_{col}'] = mse_test

    print("metrics_row:", metrics_row)
    # Append the row dictionaries as new rows in the DataFrames
    # Make sure metrics_row is a dict, not a list of dicts
    if isinstance(metrics_row, dict):
        row_df = pd.DataFrame([metrics_row])
    else:
        row_df = pd.DataFrame(metrics_row)

    metrics_df_ARX_pendulum_dt_senstivery_test_parralel = pd.concat([metrics_df_ARX_pendulum_dt_senstivery_test_parralel, row_df], ignore_index=True)


In [ ]:
metrics_df_ARX_pendulum_dt_senstivery_test_parralel

In [ ]:
# Base save folder
save_folder = 'Results/dt_tests'

# Create the directory if it doesn't exist
os.makedirs(save_folder, exist_ok=True)
# Define the file name
file_path = os.path.join(save_folder, "metrics_df_ARX_pendulum_dt_senstivery_test_parralel.csv")

# Save the DataFrame
metrics_df_ARX_pendulum_dt_senstivery_test_parralel.to_csv(file_path, index=False)



## data Senstivity

Parallel

In [ ]:
lenght=np.array([0.002,0.004,0.008,0.01,0.02 ,0.05 , 0.1 , 0.2 , 0.3 , 0.4 , 0.5 , 0.6 , 0.7 , 0.8 , 0.9,1])*len(df_case_train)
lenght=lenght.astype(int)

output_cols = ['pendulum']
#for na in range(4,na_max+1):
na=2
nb=4
nf=2

xcase = 'eta'  # xcases
input_cols=['eta' , 'eta_velocity', 'eta_acceleration']

In [ ]:
# preparing training data

dfx_train,dfy_train,yi_train = build_arx_lagged_with_scalers(
        df = df_case_train,
        input_cols  = input_cols,
        output_cols   = output_cols,
        scaler_X_func   = scaler_X_func_nov,
        scaler_y_func   = scaler_y_func_pend,
        na=na,
        nb_past=nb,
        nf_future=nf,
        test_name_col='test_name',
        y_initial_mode='zero'  # 'original' ➔ skip initial rows, 'zero' ➔ pad lags with zeros
)
    
y_train_target = dfy_train[output_cols].reset_index(drop=True)



# prepare testing data

dfx_test,dfy_test,yi_test = build_arx_lagged_with_scalers(
        df = df_case_test,
        input_cols  = input_cols,
        output_cols   = output_cols,
        scaler_X_func   = scaler_X_func_nov,
        scaler_y_func   = scaler_y_func_pend,
        na=na,
        nb_past=nb,
        nf_future=nf,
        test_name_col='test_name',
        y_initial_mode='original'  # 'original' ➔ skip initial rows, 'zero' ➔ pad lags with zeros
)


# select eta only in input features
X_train_selected_df = select_input_features(
    dfx_train.drop(columns='test_name'),
    input_type=xcase,
    keep_future=True,
    keep_past=True
)

X_test_selected_df = select_input_features(
    dfx_test.drop(columns='test_name'),
    input_type=xcase,
    keep_future=True,
    keep_past=True
)
   

In [ ]:
metrics_df_ARX_pendulum_data_test_parallel=pd.DataFrame()

In [ ]:
 
for l in lenght:
    
    # Create the pipeline model
    model = Pipeline([
            ('polynomial', PolynomialFeatures(degree=degree, interaction_only=False)),
            ('linear', LinearRegression())
        ])
        
    # prepare training data 
    print(f'-----Preprocessing case of lenght {str(l)} ----')
    # ============================
    # MEASURE CPU & MEMORY USAGE
    # ============================
    print(f'-----Training model----')
    
    model.fit(X_train_selected_df[0:l]  ,y_train_target[0:l] )
    


    # get feature names
    X_feature_names = X_train_selected_df.columns.tolist()


    print(f'-----predicting on training data----')

    # Predict on train and Testing data

    y_pred_train_scaled =model.predict(X_train_selected_df[0:l])
    
    print(f'-----predicting on Testing data----')
    
    y_pred_test_scaled = model.predict(X_test_selected_df)



    # Inverse transform to original scale
    y_pred_train = scaler_y_pend.inverse_transform(y_pred_train_scaled)
    y_pred_test = scaler_y_pend.inverse_transform(y_pred_test_scaled)


   

    # Convert predictions to DataFrames for easier handling
    y_pred_train_df = pd.DataFrame(y_pred_train, columns=output_cols)
    y_pred_test_df = pd.DataFrame(y_pred_test, columns=output_cols)


    # Get true values aligned with dfy_train and dfy_test indexes
    y_true_train =scaler_y_pend.inverse_transform(dfy_train[output_cols][0:l].reset_index(drop=True))
    y_true_train_df = pd.DataFrame(y_true_train, columns=output_cols)

    y_true_test = scaler_y_pend.inverse_transform(dfy_test[output_cols].reset_index(drop=True))
    y_true_test_df = pd.DataFrame(y_true_test, columns=output_cols)



    # save model info to dfs
    metrics_row = {
    'model_name': model_name,
    'na': na,
    'nb': nb,
    'nf': nf,
    'xcase': xcase,
    'test case': case,
    'lenght' : l
    }

    
    print(f'-----Eavluating model----')
    # Compute metrics
    if np.any(np.isnan(y_pred_train_df)) or np.any(np.isinf(y_pred_train_df)):
        print("[WARN] Train predictions have NaNs or infs.")

    for col in output_cols:
        mse_train = mean_squared_error(y_true_train_df[col], y_pred_train_df[col])
        r2_train = r2_score(y_true_train_df[col], y_pred_train_df[col])

        mse_test = mean_squared_error(y_true_test_df[col], y_pred_test_df[col])
        r2_test = r2_score(y_true_test_df[col], y_pred_test_df[col])

        # Add to metrics row
        metrics_row[f'r2_train_{col}'] = r2_train
        metrics_row[f'mse_train_{col}'] = mse_train
        metrics_row[f'r2_test_{col}'] = r2_test
        metrics_row[f'mse_test_{col}'] = mse_test

    print("metrics_row:", metrics_row)
    # Append the row dictionaries as new rows in the DataFrames
    # Make sure metrics_row is a dict, not a list of dicts
    if isinstance(metrics_row, dict):
        row_df = pd.DataFrame([metrics_row])
    else:
        row_df = pd.DataFrame(metrics_row)

    metrics_df_ARX_pendulum_data_test_parallel = pd.concat([metrics_df_ARX_pendulum_data_test_parallel, row_df], ignore_index=True)



In [ ]:
# Base save folder
save_folder = 'Results/data_tests'

# Create the directory if it doesn't exist
os.makedirs(save_folder, exist_ok=True)
# Define the file name
file_path = os.path.join(save_folder, "metrics_df_ARX_pendulum_data_test_parallel-.csv")

# Save the DataFrame
metrics_df_ARX_pendulum_data_test_parallel.to_csv(file_path, index=False)



Series

In [ ]:
metrics_df_ARX_pendulum_data_test_pseries=pd.DataFrame()

In [ ]:
 
for l in lenght:
    


    # Create the pipeline model
    model = Pipeline([
            ('polynomial', PolynomialFeatures(degree=degree, interaction_only=False)),
            ('linear', LinearRegression())
        ])
        
  
    print(f'-----Training model----')
    
    model.fit(X_train_selected_df[0:l]  ,y_train_target[0:l] )
    


    # get feature names
    X_feature_names = X_train_selected_df.columns.tolist()


    print(f'-----predicting on training data----')

    # Predict on train and Testing data
    start_time = time.perf_counter()

    y_pred_train_scaled,x_used_train = predict_recursive_series(
    model=model,
    X_df=X_train_selected_df[0:l],
    output_cols=output_cols,
    X_feature_names=X_feature_names,
    na=na
        )

    print(f'-----predicting on Testing data----')
    
    y_pred_test_scaled , x_used_test=predict_recursive_series(
    model=model,
    X_df=X_test_selected_df,
    output_cols=output_cols,
    X_feature_names=X_feature_names,
    na=na
        )



    # Inverse transform to original scale
    y_pred_train = scaler_y_pend.inverse_transform(y_pred_train_scaled)
    y_pred_test = scaler_y_pend.inverse_transform(y_pred_test_scaled)


   

    # Convert predictions to DataFrames for easier handling
    y_pred_train_df = pd.DataFrame(y_pred_train, columns=output_cols)
    y_pred_test_df = pd.DataFrame(y_pred_test, columns=output_cols)


    # Get true values aligned with dfy_train and dfy_test indexes
    y_true_train =scaler_y_pend.inverse_transform(dfy_train[output_cols][0:l].reset_index(drop=True))
    y_true_train_df = pd.DataFrame(y_true_train, columns=output_cols)

    y_true_test = scaler_y_pend.inverse_transform(dfy_test[output_cols].reset_index(drop=True))
    y_true_test_df = pd.DataFrame(y_true_test, columns=output_cols)



    # save model info to dfs
    metrics_row = {
    'model_name': model_name,
    'na': na,
    'nb': nb,
    'nf': nf,
    'xcase': xcase,
    'test case': case,
    'lenght' : l
    }

    
    print(f'-----Eavluating model----')
    # Compute metrics
    if np.any(np.isnan(y_pred_train_df)) or np.any(np.isinf(y_pred_train_df)):
        print("[WARN] Train predictions have NaNs or infs.")

    for col in output_cols:
        mse_train = mean_squared_error(y_true_train_df[col], y_pred_train_df[col])
        r2_train = r2_score(y_true_train_df[col], y_pred_train_df[col])

        mse_test = mean_squared_error(y_true_test_df[col], y_pred_test_df[col])
        r2_test = r2_score(y_true_test_df[col], y_pred_test_df[col])

        # Add to metrics row
        metrics_row[f'r2_train_{col}'] = r2_train
        metrics_row[f'mse_train_{col}'] = mse_train
        metrics_row[f'r2_test_{col}'] = r2_test
        metrics_row[f'mse_tesr_{col}'] = mse_test

    print("metrics_row:", metrics_row)
    # Append the row dictionaries as new rows in the DataFrames
    # Make sure metrics_row is a dict, not a list of dicts
    if isinstance(metrics_row, dict):
        row_df = pd.DataFrame([metrics_row])
    else:
        row_df = pd.DataFrame(metrics_row)

    metrics_df_ARX_pendulum_data_test_pseries = pd.concat([metrics_df_ARX_pendulum_data_test_pseries, row_df], ignore_index=True)


In [ ]:
# Base save folder
save_folder = 'Results/data_tests'

# Create the directory if it doesn't exist
os.makedirs(save_folder, exist_ok=True)
# Define the file name
file_path = os.path.join(save_folder, "metrics_df_ARX_pendulum_data_test_pseries-.csv")

# Save the DataFrame
metrics_df_ARX_pendulum_data_test_pseries.to_csv(file_path, index=False)



## Testing

In [ ]:
# now get the best model and predict on the test data
# Define the best model parameters
best_na = 2
best_nb = 4
best_nf = 2
best_degree = 1
best_xcase = 'eta'
best_input_cols = ['eta', 'eta_velocity', 'eta_acceleration']
best_output_cols = ['pendulum']
# load the best model
folder='ARX/saved_models/new'
#folder = 'ARX/choosen heros/pendulum'
model_name = f"ARX_pend{best_xcase}_na{best_na}_nb{best_nb}_nf{best_nf}.joblib"
# Load the model
model_path = os.path.join(folder, model_name)  
model = joblib.load(model_path)

# print all test cases
test_cases = df_test_full['test_name'].unique()
print("All test cases:")
print(test_cases)  



In [ ]:
input_cols = ['eta']  # original inputs
output_cols = ['pendulum']  # outputs

# Construct feature names in the exact order used by the data pipeline
feature_names = []

# 1. Current input signals (no lag)
feature_names += input_cols

# 2. Output lags (autoregressive part)
for lag in range(1, best_na + 1):
    for out_col in output_cols:
        feature_names.append(f"{out_col}_lag_{lag}")

# 3. Past input lags
for lag in range(1, best_nb + 1):
    for in_col in input_cols:
        feature_names.append(f"{in_col}_past_{lag}")

# 4. Future input lags
for lag in range(1, best_nf + 1):
    for in_col in input_cols:
        feature_names.append(f"{in_col}_future_{lag}")

# View the final ordered feature list
print("Feature names used in X_lagged_df:")
print(feature_names)

In [ ]:
poly = model.named_steps['polynomial']
linear = model.named_steps['linear']

# Use the names remembered from training
input_names = poly.get_feature_names_out(poly.feature_names_in_)

# Coefficients and intercept
coefficients = linear.coef_.flatten()
intercept = float(linear.intercept_)


# Build equation
equation_terms = [f"{coef:.4f} * {name}" for coef, name in zip(coefficients, input_names)]
equation = f"{intercept:.4f} + " + " + ".join(equation_terms)

print("\n📘 Model Equation:")
print(f"y = {equation}")


In [ ]:
metrics_df_test_pendulum_na2_nb4_nf2_val=pd.DataFrame()

In [ ]:
cases=df_test_full['test_name'].unique()
for case in cases:
    # Filter the test data
    df_case_test = df_test_full[df_test_full['test_name'] == case].reset_index(drop=True)
    # Prepare the test data
    dfx_test, dfy_test, yi_test = build_arx_lagged_with_scalers(
        df=df_case_test,
        input_cols=best_input_cols,
        output_cols=best_output_cols,
        scaler_X_func   = scaler_X_func_nov,
        scaler_y_func   = scaler_y_func_pend,
        na=best_na,
        nb_past=best_nb,
        nf_future=best_nf,
        test_name_col='test_name',
        y_initial_mode='original'  # 'original' ➔ skip initial rows, 'zero' ➔ pad lags with zeros
    )

    # Select only the best input features
    X_test_selected_df = select_input_features(
        dfx_test.drop(columns='test_name'),
        input_type=best_xcase,
        keep_future=True,
        keep_past=True
    )

    # get feature names from model
    X_feature_names = X_test_selected_df.columns.tolist()

    # Predict on test data
    y_pred_test_scaled, x_used_test = predict_recursive_series(
        model=model,
        X_df=X_test_selected_df,
        output_cols=best_output_cols,
        X_feature_names=X_feature_names,
        na=best_na
    )

    # Inverse transform to original scale
    y_pred_test = scaler_y_pend.inverse_transform(y_pred_test_scaled)
    # Convert to DataFrame
    y_pred_test_df = pd.DataFrame(y_pred_test, columns=best_output_cols)

    # Get true values aligned with dfy_test indexes
    y_true_test = scaler_y_pend.inverse_transform(dfy_test[best_output_cols].reset_index(drop=True))
    y_true_test_df = pd.DataFrame(y_true_test, columns=best_output_cols)

    # Compute metrics
    metrics_test = {}
    metrics_test[f'test case'] = case
    if case == training_case:
        metrics_test[f'Comments'] = 'case used for training'
   
    for col in best_output_cols:
        mse_test = mean_squared_error(y_true_test_df[col], y_pred_test_df[col])
        r2_test = r2_score(y_true_test_df[col], y_pred_test_df[col])

        
        metrics_test[f'r2_test_{col}'] = r2_test
        metrics_test[f'mse_test_{col}'] = mse_test
    
    print("metrics:", metrics_test)
    # Append the row dictionaries as new rows in the DataFrame
    metrics_df_test_pendulum_na2_nb4_nf2_val = pd.concat([metrics_df_test_pendulum_na2_nb4_nf2_val, pd.DataFrame([metrics_test])], ignore_index=True)
        # Save true and predicted DataFrames to CSV files
    y_true_test_df.to_csv(f'Results/ARX/Testing/pendulum/y_true_test_pitch_{case}.csv', index=False)
    y_pred_test_df.to_csv(f'Results/ARX/Testing/pendulum/y_pred_test_df{case}.csv', index=False)

    

In [ ]:
# saving the metrics DataFrame
metrics_df_test_pendulum_na2_nb4_nf2_val.to_csv('ARX/choosen heros/metrics_df_test_pendulum_na2_nb4_nf2_val.csv', index=False)

## Noisy data test

In [ ]:
# now get the best model and predict on the test data
# Define the best model parameters
best_na = 2
best_nb = 4
best_nf = 2
best_degree = 1
best_xcase = 'eta'
best_input_cols = ['eta', 'eta_velocity', 'eta_acceleration']
best_output_cols = ['pendulum']
# load the best model
folder='ARX/saved_models/new'
#folder = 'ARX/choosen heros/pendulum'
model_name = f"ARX_pend{best_xcase}_na{best_na}_nb{best_nb}_nf{best_nf}.joblib"
# Load the model
model_path = os.path.join(folder, model_name)  
model = joblib.load(model_path)



In [ ]:
# load noisy data
df_case_test_noisy=pd.read_csv('Results/df_case_test_noisy.csv')
df_case_test_noisy

In [ ]:
# prepare the test data
dfx_test, dfy_test, yi_test = build_arx_lagged_with_scalers(
    df=df_case_test_noisy,
    input_cols=best_input_cols,
    output_cols=best_output_cols,
    scaler_X_func   = scaler_X_func_nov,
    scaler_y_func   = scaler_y_func_pend,
    na=best_na,
    nb_past=best_nb,
    nf_future=best_nf,
    test_name_col='test_name',
    y_initial_mode='original'  # 'original' ➔ skip initial rows, 'zero' ➔ pad lags with zeros
)

# Select only the best input features
X_test_selected_df = select_input_features(
    dfx_test.drop(columns='test_name'),
    input_type=best_xcase,
    keep_future=True,
    keep_past=True
)

# get feature names from model
X_feature_names = X_test_selected_df.columns.tolist()

# Predict on test data
y_pred_test_scaled, x_used_test = predict_recursive_series(
    model=model,
    X_df=X_test_selected_df,
    output_cols=best_output_cols,
    X_feature_names=X_feature_names,
    na=best_na
)

# Inverse transform to original scale
y_pred_test = scaler_y_pend.inverse_transform(y_pred_test_scaled)
# Convert to DataFrame
y_pred_test_df = pd.DataFrame(y_pred_test, columns=best_output_cols)

# Get true values aligned with dfy_test indexes
y_true_test = scaler_y_pend.inverse_transform(dfy_test[best_output_cols].reset_index(drop=True))
y_true_test_df = pd.DataFrame(y_true_test, columns=best_output_cols)

# Compute metrics
metrics_test = {}
metrics_test[f'test case'] = case


mse_test = mean_squared_error(y_true_test_df['pendulum'], y_pred_test_df['pendulum'])
r2_test = r2_score(y_true_test_df['pendulum'], y_pred_test_df['pendulum'])


metrics_test[f'r2_test_pendulum'] = r2_test
metrics_test[f'mse_test_pendulum'] = mse_test

print("metrics:", metrics_test)

In [ ]:
y_true_test_df.to_csv(f'Results/noisy test/ARX_pendulum_true.csv', index=False)
y_pred_test_df.to_csv(f'Results/noisy test/ARX_pendulum_pred.csv', index=False)

# Computation time and emisions caculations

In [ ]:
import codecarbon
print(codecarbon.__version__)

from codecarbon import EmissionsTracker




We adjust the data sizes to be devisible by 64 (like in lstm case) for a fair comparison

In [ ]:
batch_size=64

def adjust_for_batch_df(df, y):
    # Calculate the number of rows that are divisible by the batch size
    n = (df.shape[0] // batch_size) * batch_size
    
    # Adjust DataFrame and target variable
    df_adjusted = df.iloc[:n]  # Select the rows to match the batch size
    y_adjusted = y[:n]         # Adjust the target variable in the same way
    
    return df_adjusted, y_adjusted


## Heave

In [ ]:
# define best lag values 
na  = 2
nb= 2
nf=7
degree = 1
xcase = 'eta'  # xcases
input_cols=['eta' , 'eta_velocity', 'eta_acceleration']
output_cols = ['heave']
# Define the scaling functions (they apply the .transform)
case='Tp6p8s_Hs2m'
case_test='Tp6p8s_Hs1m'
df_case_train=df_train_full[df_train_full['test_name']==case].reset_index(drop=True)
df_case_test=df_train_full[df_train_full['test_name']==case_test].reset_index(drop=True)



In [ ]:
# prepare training data
dfx_train,dfy_train,yi_train = build_arx_lagged_with_scalers(
        df = df_case_train,
        input_cols  = input_cols,
        output_cols   = output_cols,
        scaler_X_func   = scaler_X_func_nov,
        scaler_y_func   = scaler_y_func_heave,
        na=na,
        nb_past=nb,
        nf_future=nf,
        test_name_col='test_name',
        y_initial_mode='zero'  # 'original' ➔ skip initial rows, 'zero' ➔ pad lags with zeros
)
    
y_train_target = dfy_train[output_cols].reset_index(drop=True)
# select eta only in input features
X_train_selected_df = select_input_features(
    dfx_train.drop(columns='test_name'),
    input_type=xcase,
    keep_future=True,
    keep_past=True
)

X_feature_names = X_train_selected_df.columns.tolist()


X_train_selected_df, y_train_target = adjust_for_batch_df(X_train_selected_df, y_train_target)



In [ ]:
process = psutil.Process()
start_time = time.time()  # Start time for measurement
initial_memory = process.memory_info().rss / (1024 ** 2)  # Memory in MB
# Create the pipeline model
model = Pipeline([
        ('polynomial', PolynomialFeatures(degree=degree, interaction_only=False)),
        ('linear', LinearRegression())
    ])
    
#fit the model
model.fit(X_train_selected_df  ,y_train_target  )

# Measure memory usage and time after prediction
end_time = time.time()  # End time for measurement
final_memory = process.memory_info().rss / (1024 ** 2)  # Memory in MB
# Print results
print(f"Time taken : {end_time - start_time} seconds")
print(f"Memory used: {final_memory - initial_memory} MB")

In [ ]:
tracker = EmissionsTracker(
    output_dir="Results/co2",       # Custom folder
    output_file="arx_heave_train.csv"    # Custom filename
)

tracker.start()

# Define and train model
model = Pipeline([
    ('polynomial', PolynomialFeatures(degree=degree, interaction_only=False)),
    ('linear', LinearRegression())
])

model.fit(X_train_selected_df  ,y_train_target  )

# Stop the tracker and retrieve the estimated emissions
emissions = tracker.stop()


In [ ]:
# prepare Testing data
dfx_test,dfy_test,yi_test = build_arx_lagged_with_scalers(
        df = df_case_test,
        input_cols  = input_cols,
        output_cols   = output_cols,
        scaler_X_func   = scaler_X_func_nov,
        scaler_y_func   = scaler_y_func_heave,
        na=na,
        nb_past=nb,
        nf_future=nf,
        test_name_col='test_name',
        y_initial_mode='zero'  # 'original' ➔ skip initial rows, 'zero' ➔ pad lags with zeros
)
    
y_test_target = dfy_test[output_cols].reset_index(drop=True)
# select eta only in input features
X_test_selected_df = select_input_features(
    dfx_test.drop(columns='test_name'),
    input_type=xcase,
    keep_future=True,
    keep_past=True
)


X_test_selected_df, y_test_target = adjust_for_batch_df(X_test_selected_df, y_test_target)


In [ ]:
process = psutil.Process()
start_time = time.time()  # Start time for measurement
initial_memory = process.memory_info().rss / (1024 ** 2)  # Memory in MB
# parralel prediction 

y_pred_heave_test=model.predict(X_test_selected_df)



# Measure memory usage and time after prediction
end_time = time.time()  # End time for measurement
final_memory = process.memory_info().rss / (1024 ** 2)  # Memory in MB
# Print results
print(f"Time taken : {end_time - start_time} seconds")
print(f"Memory used: {final_memory - initial_memory} MB")

In [ ]:
tracker = EmissionsTracker(
    output_dir="Results/co2",       # Custom folder
    output_file="arx_heave_parralel_pred.csv"    # Custom filename
)
tracker.start()
# parralel prediction 
y_pred_heave_test=model.predict(X_test_selected_df)


# Stop the tracker and retrieve the estimated emissions
emissions = tracker.stop()


In [ ]:
process = psutil.Process()
start_time = time.time()  # Start time for measurement
initial_memory = process.memory_info().rss / (1024 ** 2)  # Memory in MB
# series prediction 

# Run prediction process 
y_pred_test_series_case_compare_heave, x_used_test = predict_recursive_series(
    model=model,
    X_df=X_test_selected_df,
    output_cols=output_cols,
    X_feature_names=X_feature_names,
    na=na
)



# Measure memory usage and time after prediction
end_time = time.time()  # End time for measurement
final_memory = process.memory_info().rss / (1024 ** 2)  # Memory in MB
# Print results
print(f"Time taken : {end_time - start_time} seconds")
print(f"Memory used: {final_memory - initial_memory} MB")

In [ ]:
tracker = EmissionsTracker(
    output_dir="Results/co2",       # Custom folder
    output_file="arx_heave_series.csv"    # Custom filename
)
tracker.start()

# series prediction 

# Run prediction process 
y_pred_test_series_case_compare_heave, x_used_test = predict_recursive_series(
    model=model,
    X_df=X_test_selected_df,
    output_cols=output_cols,
    X_feature_names=X_feature_names,
    na=na
)



# Stop the tracker and retrieve the estimated emissions
emissions = tracker.stop()

## Pitch

In [ ]:
# define best lag values 
na  = 2
nb= 2
nf=7
degree = 1
xcase = 'eta'  # xcases
input_cols=['eta' , 'eta_velocity', 'eta_acceleration']
output_cols = ['pitch']
# Define the scaling functions (they apply the .transform)
case='Tp6p8s_Hs2m'
case_test='Tp6p8s_Hs1m'
df_case_train=df_train_full[df_train_full['test_name']==case].reset_index(drop=True)
df_case_test=df_train_full[df_train_full['test_name']==case_test].reset_index(drop=True)


In [ ]:
# prepare training data
dfx_train,dfy_train,yi_train = build_arx_lagged_with_scalers(
        df = df_case_train,
        input_cols  = input_cols,
        output_cols   = output_cols,
        scaler_X_func   = scaler_X_func_nov,
        scaler_y_func   = scaler_y_func_pitch,
        na=na,
        nb_past=nb,
        nf_future=nf,
        test_name_col='test_name',
        y_initial_mode='zero'  # 'original' ➔ skip initial rows, 'zero' ➔ pad lags with zeros
)
    
y_train_target = dfy_train[output_cols].reset_index(drop=True)
# select eta only in input features
X_train_selected_df = select_input_features(
    dfx_train.drop(columns='test_name'),
    input_type=xcase,
    keep_future=True,
    keep_past=True
)

X_feature_names = X_train_selected_df.columns.tolist()
X_train_selected_df, y_train_target = adjust_for_batch_df(X_train_selected_df, y_train_target)


In [ ]:
process = psutil.Process()
start_time = time.time()  # Start time for measurement
initial_memory = process.memory_info().rss / (1024 ** 2)  # Memory in MB
# Create the pipeline model
model = Pipeline([
        ('polynomial', PolynomialFeatures(degree=degree, interaction_only=False)),
        ('linear', LinearRegression())
    ])
    
# fit the model
model.fit(X_train_selected_df  ,y_train_target  )

# Measure memory usage and time after prediction
end_time = time.time()  # End time for measurement
final_memory = process.memory_info().rss / (1024 ** 2)  # Memory in MB
# Print results
print(f"Time taken : {end_time - start_time} seconds")
print(f"Memory used: {final_memory - initial_memory} MB")

In [ ]:
tracker = EmissionsTracker(
    output_dir="Results/co2",       # Custom folder
    output_file="arx_pitch_train.csv"    # Custom filename
)

tracker.start()

# Define and train your model
model = Pipeline([
    ('polynomial', PolynomialFeatures(degree=degree, interaction_only=False)),
    ('linear', LinearRegression())
])

model.fit(X_train_selected_df  ,y_train_target  )

# Stop the tracker and retrieve the estimated emissions
emissions = tracker.stop()


In [ ]:
# prepare Testing data
dfx_test,dfy_test,yi_test = build_arx_lagged_with_scalers(
        df = df_case_test,
        input_cols  = input_cols,
        output_cols   = output_cols,
        scaler_X_func   = scaler_X_func_nov,
        scaler_y_func   = scaler_y_func_pitch,
        na=na,
        nb_past=nb,
        nf_future=nf,
        test_name_col='test_name',
        y_initial_mode='zero'  # 'original' ➔ skip initial rows, 'zero' ➔ pad lags with zeros
)
    
y_test_target = dfy_test[output_cols].reset_index(drop=True)
# select eta only in input features
X_test_selected_df = select_input_features(
    dfx_test.drop(columns='test_name'),
    input_type=xcase,
    keep_future=True,
    keep_past=True
)

X_test_selected_df, y_test_target = adjust_for_batch_df(X_test_selected_df, y_test_target)

In [ ]:
process = psutil.Process()
start_time = time.time()  # Start time for measurement
initial_memory = process.memory_info().rss / (1024 ** 2)  # Memory in MB
# parralel prediction 

y_pred_pitch_test=model.predict(X_test_selected_df)



# Measure memory usage and time after prediction
end_time = time.time()  # End time for measurement
final_memory = process.memory_info().rss / (1024 ** 2)  # Memory in MB
# Print results
print(f"Time taken : {end_time - start_time} seconds")
print(f"Memory used: {final_memory - initial_memory} MB")

In [ ]:
tracker = EmissionsTracker(
    output_dir="Results/co2",       # Custom folder
    output_file="arx_pitch_parralel_pred.csv"    # Custom filename
)
tracker.start()
# parralel prediction 
y_pred_pitch_test=model.predict(X_test_selected_df)


# Stop the tracker and retrieve the estimated emissions
emissions = tracker.stop()


In [ ]:
process = psutil.Process()
start_time = time.time()  # Start time for measurement
initial_memory = process.memory_info().rss / (1024 ** 2)  # Memory in MB
# series prediction 

# Run prediction process 
y_pred_test_series_case_compare_heave, x_used_test = predict_recursive_series(
    model=model,
    X_df=X_test_selected_df,
    output_cols=output_cols,
    X_feature_names=X_feature_names,
    na=na
)



# Measure memory usage and time after prediction
end_time = time.time()  # End time for measurement
final_memory = process.memory_info().rss / (1024 ** 2)  # Memory in MB
# Print results
print(f"Time taken : {end_time - start_time} seconds")
print(f"Memory used: {final_memory - initial_memory} MB")

In [ ]:
tracker = EmissionsTracker(
    output_dir="Results/co2",       # Custom folder
    output_file="arx_pitch_series.csv"    # Custom filename
)
tracker.start()

# series prediction 

# Run prediction process 
y_pred_test_series_case_compare_heave, x_used_test = predict_recursive_series(
    model=model,
    X_df=X_test_selected_df,
    output_cols=output_cols,
    X_feature_names=X_feature_names,
    na=na
)



# Stop the tracker and retrieve the estimated emissions
emissions = tracker.stop()

## pendulum

In [ ]:
# define best lag values 
na  = 2
nb= 4
nf=2
degree = 1
xcase = 'eta'  # xcases
input_cols=['eta' , 'eta_velocity', 'eta_acceleration']
output_cols = ['pendulum']
# Define the scaling functions (they apply the .transform)
case='Tp6p8s_Hs2m'
case_test='Tp6p8s_Hs1m'
df_case_train=df_train_full[df_train_full['test_name']==case].reset_index(drop=True)
df_case_test=df_train_full[df_train_full['test_name']==case_test].reset_index(drop=True)


In [ ]:
# prepare training data
dfx_train,dfy_train,yi_train = build_arx_lagged_with_scalers(
        df = df_case_train,
        input_cols  = input_cols,
        output_cols   = output_cols,
        scaler_X_func   = scaler_X_func_nov,
        scaler_y_func   = scaler_y_func_pend,
        na=na,
        nb_past=nb,
        nf_future=nf,
        test_name_col='test_name',
        y_initial_mode='zero'  # 'original' ➔ skip initial rows, 'zero' ➔ pad lags with zeros
)
    
y_train_target = dfy_train[output_cols].reset_index(drop=True)
# select eta only in input features
X_train_selected_df = select_input_features(
    dfx_train.drop(columns='test_name'),
    input_type=xcase,
    keep_future=True,
    keep_past=True
)

X_feature_names = X_train_selected_df.columns.tolist()

X_train_selected_df, y_train_target = adjust_for_batch_df(X_train_selected_df, y_train_target)


In [ ]:
process = psutil.Process()
start_time = time.time()  # Start time for measurement
initial_memory = process.memory_info().rss / (1024 ** 2)  # Memory in MB
# Create the pipeline model
model = Pipeline([
        ('polynomial', PolynomialFeatures(degree=degree, interaction_only=False)),
        ('linear', LinearRegression())
    ])
    
# fit the model
model.fit(X_train_selected_df  ,y_train_target  )

# Measure memory usage and time after prediction
end_time = time.time()  # End time for measurement
final_memory = process.memory_info().rss / (1024 ** 2)  # Memory in MB
# Print results
print(f"Time taken : {end_time - start_time} seconds")
print(f"Memory used: {final_memory - initial_memory} MB")

In [ ]:
tracker = EmissionsTracker(
    output_dir="Results/co2",       # Custom folder
    output_file="arx_pendulum_train.csv"    # Custom filename
)

tracker.start()

# Define and train model
model = Pipeline([
    ('polynomial', PolynomialFeatures(degree=degree, interaction_only=False)),
    ('linear', LinearRegression())
])

model.fit(X_train_selected_df  ,y_train_target  )

# Stop the tracker and retrieve the estimated emissions
emissions = tracker.stop()


In [ ]:
# prepare Testing data
dfx_test,dfy_test,yi_test = build_arx_lagged_with_scalers(
        df = df_case_test,
        input_cols  = input_cols,
        output_cols   = output_cols,
        scaler_X_func   = scaler_X_func_nov,
        scaler_y_func   = scaler_y_func_pend,
        na=na,
        nb_past=nb,
        nf_future=nf,
        test_name_col='test_name',
        y_initial_mode='zero'  # 'original' ➔ skip initial rows, 'zero' ➔ pad lags with zeros
)
    
y_test_target = dfy_test[output_cols].reset_index(drop=True)
# select eta only in input features
X_test_selected_df = select_input_features(
    dfx_test.drop(columns='test_name'),
    input_type=xcase,
    keep_future=True,
    keep_past=True
)
X_test_selected_df, y_test_target = adjust_for_batch_df(X_test_selected_df, y_test_target)

In [ ]:
process = psutil.Process()
start_time = time.time()  # Start time for measurement
initial_memory = process.memory_info().rss / (1024 ** 2)  # Memory in MB
# parralel prediction 

y_pred_pendulum_test=model.predict(X_test_selected_df)



# Measure memory usage and time after prediction
end_time = time.time()  # End time for measurement
final_memory = process.memory_info().rss / (1024 ** 2)  # Memory in MB
# Print results
print(f"Time taken : {end_time - start_time} seconds")
print(f"Memory used: {final_memory - initial_memory} MB")

In [ ]:
tracker = EmissionsTracker(
    output_dir="Results/co2",       # Custom folder
    output_file="arx_pendulum_parralel_pred.csv"    # Custom filename
)
tracker.start()

# parralel prediction
y_pred_pendulum_test=model.predict(X_test_selected_df)


# Stop the tracker and retrieve the estimated emissions
emissions = tracker.stop()


In [ ]:
process = psutil.Process()
start_time = time.time()  # Start time for measurement
initial_memory = process.memory_info().rss / (1024 ** 2)  # Memory in MB
# series prediction 

# Run prediction process 
y_pred_test_series_case_compare_heave, x_used_test = predict_recursive_series(
    model=model,
    X_df=X_test_selected_df,
    output_cols=output_cols,
    X_feature_names=X_feature_names,
    na=na
)



# Measure memory usage and time after prediction
end_time = time.time()  # End time for measurement
final_memory = process.memory_info().rss / (1024 ** 2)  # Memory in MB
# Print results
print(f"Time taken : {end_time - start_time} seconds")
print(f"Memory used: {final_memory - initial_memory} MB")

In [ ]:
tracker = EmissionsTracker(
    output_dir="Results/co2",       # Custom folder
    output_file="arx_pendulum_series.csv"    # Custom filename
)
tracker.start()

# series prediction 

# Run prediction process 
y_pred_test_series_case_compare_heave, x_used_test = predict_recursive_series(
    model=model,
    X_df=X_test_selected_df,
    output_cols=output_cols,
    X_feature_names=X_feature_names,
    na=na
)



# Stop the tracker and retrieve the estimated emissions
emissions = tracker.stop()